In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import altair as alt
from requests.utils import quote
import os
from github import Github

In [2]:
end_date = "2019-06-01"
start_date = "2019-05-01"
renderer = "jupyterlab"

In [3]:
# Parameters
end_date = "2019-06-20"
start_date = "2019-05-21"
renderer = "jupyterlab"


In [4]:
# Create the renderer we want
alt.renderers.enable(renderer)

RendererRegistry.enable('jupyterlab')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# GitHub
def run_github_query(query, github_base="https://api.github.com/search/issues"):
    gh = Github(os.environ["GITHUB_ACCESS_TOKEN"])
    resp = gh.search_issues(query)
    data = []
    for ii in range(0, 10):
        items = resp.get_page(ii)
        if len(items) > 0:
            items_list = [item.raw_data for item in items]
            data.append(pd.DataFrame.from_records(items_list))
        else:
            break
    return pd.concat(data)


# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (93 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (29 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (23 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (13 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Creating a new Binder-at-home tool](https://discourse.jupyter.org/t/creating-a-new-binder-at-home-tool) (13 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (93 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (30 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (30 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (29 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (23 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

<VegaLite 2 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Bitcoin mining (abuse/security)](https://discourse.jupyter.org/t/bitcoin-mining-abuse-security) (16 posts)
* [I get `Read timed out` errors on when pushing image OVH](https://discourse.jupyter.org/t/i-get-read-timed-out-errors-on-when-pushing-image-ovh) (7 posts)
* [Extension that is notified on notebook open event](https://discourse.jupyter.org/t/extension-that-is-notified-on-notebook-open-event) (6 posts)
* [Too many redirects - Keycloak Authentication](https://discourse.jupyter.org/t/too-many-redirects-keycloak-authentication) (3 posts)
* [JupyterHub, Binder workshop](https://discourse.jupyter.org/t/jupyterhub-binder-workshop) (3 posts)
* [Welcoming our new JupyterHub team member: Sarah!](https://discourse.jupyter.org/t/welcoming-our-new-jupyterhub-team-member-sarah) (3 posts)
* [Document domain registration for JupyterHub](https://discourse.jupyter.org/t/document-domain-registration-for-jupyterhub) (3 posts)
* [0.5 G for memory limit?](https://discourse.jupyter.org/t/0-5-g-for-memory-limit) (3 posts)
* [Quick question on jupyterlab doc regarding &lt;lab-location&gt;](https://discourse.jupyter.org/t/quick-question-on-jupyterlab-doc-regarding-lab-location) (3 posts)
* [Mybinder failing to load Jupyter widget extension](https://discourse.jupyter.org/t/mybinder-failing-to-load-jupyter-widget-extension) (2 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

<VegaLite 2 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)

In [14]:
from datetime import timedelta


orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets"]
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-21** to **2019-06-20**

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [15]:
# Update data
merged = []
for org in tqdm(orgs):
    query = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    this_merged = run_github_query(query)
    this_merged['org'] = org
    merged.append(this_merged)
merged = pd.concat(merged)
merged['repo'] = merged['repository_url'].map(lambda a: a.split('/')[-1])

# Count PRs by org/repo
prs_by_repo = merged.groupby(['org', 'repo']).count()['body'].reset_index().sort_values(['org', 'body'], ascending=False)

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:47<02:22, 47.62s/it]

 50%|█████     | 2/4 [01:29<01:31, 45.91s/it]

 75%|███████▌  | 3/4 [02:15<00:46, 46.00s/it]

100%|██████████| 4/4 [02:32<00:00, 37.27s/it]

In [16]:
alt.Chart(data=prs_by_repo, title="Merged PRs in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='body',
    color='org'
)

<VegaLite 2 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


### A list of merged PRs by project

In [17]:
markdown = []
for org, idata in merged.groupby('org'):
    markdown.append(f"#### {org}")
    markdown.append("")
    for repourl, prs in idata.groupby('repository_url'):
        reponame, repo = repourl.split('/')[-2:]
        markdown.append(f"##### [{reponame}/{repo}](https://github.com/{reponame}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['user']['login']
            user_url = pr['user']['html_url']
            pr_number = pr['number']
            pr_html = pr['html_url']
            pr_title = pr['title']

            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[{user_name}]({user_url})**"
            markdown.append(text)
        markdown.append('')
    markdown.append('')

Markdown('\n'.join(markdown))

#### jupyter

##### [jupyter/docker-stacks](https://github.com/jupyter/docker-stacks)
* [(#876)](https://github.com/jupyter/docker-stacks/pull/876): _Add `Specific Docker Image Options` for jupyter spark_ by **[eromoe](https://github.com/eromoe)**

##### [jupyter/enterprise_gateway](https://github.com/jupyter/enterprise_gateway)
* [(#692)](https://github.com/jupyter/enterprise_gateway/pull/692): _More build adjustments - use travis_retry to reduce intermittency_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#690)](https://github.com/jupyter/enterprise_gateway/pull/690): _Update link to use CRAN installation for Apache Spark_ by **[akchinSTC](https://github.com/akchinSTC)**
* [(#689)](https://github.com/jupyter/enterprise_gateway/pull/689): _Attempt to address build flakiness_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#688)](https://github.com/jupyter/enterprise_gateway/pull/688): _Fix scoping issues relative to python kernels on Spark_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#686)](https://github.com/jupyter/enterprise_gateway/pull/686): _Fix interrupts on intensive operations_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#685)](https://github.com/jupyter/enterprise_gateway/pull/685): _Allow port overrides in Kubernetes deployments_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#680)](https://github.com/jupyter/enterprise_gateway/pull/680): _Docs Enhancement (part 2)_ by **[echarles](https://github.com/echarles)**
* [(#679)](https://github.com/jupyter/enterprise_gateway/pull/679): _Update docs with english term for kernels_ by **[lresende](https://github.com/lresende)**
* [(#677)](https://github.com/jupyter/enterprise_gateway/pull/677): _Docs enhancements_ by **[echarles](https://github.com/echarles)**
* [(#675)](https://github.com/jupyter/enterprise_gateway/pull/675): _GitHub issue/pull request template_ by **[lresende](https://github.com/lresende)**
* [(#674)](https://github.com/jupyter/enterprise_gateway/pull/674): _Update platforms.html_ by **[philip-p-jones](https://github.com/philip-p-jones)**
* [(#673)](https://github.com/jupyter/enterprise_gateway/pull/673): _Fix container termination signal handling in EG container_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#666)](https://github.com/jupyter/enterprise_gateway/pull/666): _Makefiles: change publish targets to push, use --no-cache on new publish-image target_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#645)](https://github.com/jupyter/enterprise_gateway/pull/645): _Refactor KernelSessionManager to accommodate various persistence methods_ by **[kevin-bates](https://github.com/kevin-bates)**
* [(#607)](https://github.com/jupyter/enterprise_gateway/pull/607): _Add support for multiple YARN resource managers_ by **[kevin-bates](https://github.com/kevin-bates)**

##### [jupyter/jupyter](https://github.com/jupyter/jupyter)
* [(#427)](https://github.com/jupyter/jupyter/pull/427): _May Community Call notes_ by **[Zsailer](https://github.com/Zsailer)**

##### [jupyter/jupyter-book](https://github.com/jupyter/jupyter-book)
* [(#218)](https://github.com/jupyter/jupyter-book/pull/218): _version bump to dev_ by **[choldgraf](https://github.com/choldgraf)**
* [(#217)](https://github.com/jupyter/jupyter-book/pull/217): _version bump for bugfix_ by **[choldgraf](https://github.com/choldgraf)**
* [(#216)](https://github.com/jupyter/jupyter-book/pull/216): _fix doc mismatch for "make build"_ by **[thammegowda](https://github.com/thammegowda)**
* [(#215)](https://github.com/jupyter/jupyter-book/pull/215): _make scripts dir as a module, to be included by `find_packages()` of setuptools_ by **[thammegowda](https://github.com/thammegowda)**
* [(#209)](https://github.com/jupyter/jupyter-book/pull/209): _add jupyter book to template requirements_ by **[choldgraf](https://github.com/choldgraf)**
* [(#207)](https://github.com/jupyter/jupyter-book/pull/207): _updating hidecode tag word and allowing total HTML removal_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyter/jupyter_client](https://github.com/jupyter/jupyter_client)
* [(#444)](https://github.com/jupyter/jupyter_client/pull/444): _Update wrapper kernel docs_ by **[blink1073](https://github.com/blink1073)**
* [(#440)](https://github.com/jupyter/jupyter_client/pull/440): _Add xeus to the documentation on how to write a kernel for Jupyter_ by **[martinRenou](https://github.com/martinRenou)**
* [(#438)](https://github.com/jupyter/jupyter_client/pull/438): _Backport PR #437: Avoid kernel failures with multiple processes_ by **[alexrudy](https://github.com/alexrudy)**

##### [jupyter/jupyter_core](https://github.com/jupyter/jupyter_core)
* [(#155)](https://github.com/jupyter/jupyter_core/pull/155): _Update changelog for 4.5.0_ by **[takluyver](https://github.com/takluyver)**
* [(#154)](https://github.com/jupyter/jupyter_core/pull/154): _Fix test for jupyter_runtime_dir without XDG_RUNTIME_DIR_ by **[takluyver](https://github.com/takluyver)**
* [(#153)](https://github.com/jupyter/jupyter_core/pull/153): _Start writing changelog for 4.5.0_ by **[takluyver](https://github.com/takluyver)**
* [(#149)](https://github.com/jupyter/jupyter_core/pull/149): _Respect the raise_config_file_errors trait and re-raise if applicable._ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#148)](https://github.com/jupyter/jupyter_core/pull/148): _Removed mechanism that change $PATH_ by **[xmnlab](https://github.com/xmnlab)**
* [(#143)](https://github.com/jupyter/jupyter_core/pull/143): _Stop using XDG_RUNTIME_DIR_ by **[takluyver](https://github.com/takluyver)**
* [(#133)](https://github.com/jupyter/jupyter_core/pull/133): _Include LICENSE file in wheels_ by **[toddrme2178](https://github.com/toddrme2178)**

##### [jupyter/jupyter_server](https://github.com/jupyter/jupyter_server)
* [(#54)](https://github.com/jupyter/jupyter_server/pull/54): _Remove notebook templates and add basic server templates._ by **[Zsailer](https://github.com/Zsailer)**
* [(#49)](https://github.com/jupyter/jupyter_server/pull/49): _Add localhost to list of default local_hostnames_ by **[aschlaep](https://github.com/aschlaep)**

##### [jupyter/nb2kg](https://github.com/jupyter/nb2kg)
* [(#38)](https://github.com/jupyter/nb2kg/pull/38): _Ensure request-timeout is > launch-timeout + pad_ by **[kevin-bates](https://github.com/kevin-bates)**

##### [jupyter/nbconvert](https://github.com/jupyter/nbconvert)
* [(#1051)](https://github.com/jupyter/nbconvert/pull/1051): _fix delimiter in config docs_ by **[t-makaro](https://github.com/t-makaro)**
* [(#1050)](https://github.com/jupyter/nbconvert/pull/1050): _Fix RTD_ by **[t-makaro](https://github.com/t-makaro)**
* [(#1048)](https://github.com/jupyter/nbconvert/pull/1048): _Revamp configuration documentation_ by **[t-makaro](https://github.com/t-makaro)**
* [(#1046)](https://github.com/jupyter/nbconvert/pull/1046): _Add issue templates_ by **[t-makaro](https://github.com/t-makaro)**
* [(#1042)](https://github.com/jupyter/nbconvert/pull/1042): _simplify the function signature for preprocess()_ by **[afeld](https://github.com/afeld)**
* [(#1040)](https://github.com/jupyter/nbconvert/pull/1040): _Set flag to not always stop kernel execution on errors_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1039)](https://github.com/jupyter/nbconvert/pull/1039): _Fix LaTeX exporting '?' for non-ascii title_ by **[imtsuki](https://github.com/imtsuki)**
* [(#1036)](https://github.com/jupyter/nbconvert/pull/1036): _Fix selection of mimetype when converting to HTML_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1034)](https://github.com/jupyter/nbconvert/pull/1034): _Added instructions for bumping the version forward when releasing_ by **[MSeal](https://github.com/MSeal)**
* [(#1030)](https://github.com/jupyter/nbconvert/pull/1030): _Fix Testing on Windows_ by **[t-makaro](https://github.com/t-makaro)**
* [(#1029)](https://github.com/jupyter/nbconvert/pull/1029): _Correct a few typos_ by **[t-makaro](https://github.com/t-makaro)**
* [(#1017)](https://github.com/jupyter/nbconvert/pull/1017): _Disable IPython History in executing preprocessor_ by **[alexrudy](https://github.com/alexrudy)**

##### [jupyter/nbdime](https://github.com/jupyter/nbdime)
* [(#477)](https://github.com/jupyter/nbdime/pull/477): _Use config for webapps_ by **[vidartf](https://github.com/vidartf)**

##### [jupyter/nbgrader](https://github.com/jupyter/nbgrader)
* [(#1152)](https://github.com/jupyter/nbgrader/pull/1152): _Rename 'nbgrader fetch' to 'nbgrader fetch_assignment'_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1151)](https://github.com/jupyter/nbgrader/pull/1151): _Rename 'nbgrader release' to 'nbgrader release_assignment'_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1147)](https://github.com/jupyter/nbgrader/pull/1147): _Add test to ensure that db upgrade succeeds before running assign_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1145)](https://github.com/jupyter/nbgrader/pull/1145): _Rename nbgrader feedback to nbgrader generate_feedback_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1140)](https://github.com/jupyter/nbgrader/pull/1140): _A few more updates to the docs for multiple classes_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1139)](https://github.com/jupyter/nbgrader/pull/1139): _Additional docs sanitization_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1138)](https://github.com/jupyter/nbgrader/pull/1138): _Ensure that cell type changes result in valid nbgrader metadata_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1137)](https://github.com/jupyter/nbgrader/pull/1137): _Rename "nbgrader assign" to "nbgrader generate_assignment"_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1135)](https://github.com/jupyter/nbgrader/pull/1135): _section on grading in docker container_ by **[gertingold](https://github.com/gertingold)**
* [(#1131)](https://github.com/jupyter/nbgrader/pull/1131): _Better support for multiple classes_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1127)](https://github.com/jupyter/nbgrader/pull/1127): _Better documentation of nbgrader_config.py_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1126)](https://github.com/jupyter/nbgrader/pull/1126): _Remove the third party resources page_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1125)](https://github.com/jupyter/nbgrader/pull/1125): _Check that the course directory is a subdirectory of the notebook dir_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1124)](https://github.com/jupyter/nbgrader/pull/1124): _Only run nbextensions tests on oldest and newest versions of python_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1123)](https://github.com/jupyter/nbgrader/pull/1123): _Ensure course directory root path has no trailing slashes_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1122)](https://github.com/jupyter/nbgrader/pull/1122): _Fix incorrect usage of Exchange.course_id_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1121)](https://github.com/jupyter/nbgrader/pull/1121): _Fix logfile_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1120)](https://github.com/jupyter/nbgrader/pull/1120): _Integrate feedback distribution within nbgrader_ by **[danielmaitre](https://github.com/danielmaitre)**
* [(#1119)](https://github.com/jupyter/nbgrader/pull/1119): _added a sanatizing step to the doc creation. _ by **[danielmaitre](https://github.com/danielmaitre)**
* [(#1118)](https://github.com/jupyter/nbgrader/pull/1118): _Integrate course_id into the api and apps_ by **[damianavila](https://github.com/damianavila)**
* [(#1116)](https://github.com/jupyter/nbgrader/pull/1116): _Autograde & Assign: create missing students/assignments by default_ by **[takluyver](https://github.com/takluyver)**
* [(#1115)](https://github.com/jupyter/nbgrader/pull/1115): _Fix typo in tmp filename prefix in conftest.py_ by **[slel](https://github.com/slel)**
* [(#1114)](https://github.com/jupyter/nbgrader/pull/1114): _Documentation for multiple classes_ by **[sigurdurb](https://github.com/sigurdurb)**
* [(#1113)](https://github.com/jupyter/nbgrader/pull/1113): _Add a course list extension that shows all courses an instructor can manage_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1112)](https://github.com/jupyter/nbgrader/pull/1112): _Locate all configurable classes for generate_config subcommand_ by **[takluyver](https://github.com/takluyver)**
* [(#1111)](https://github.com/jupyter/nbgrader/pull/1111): _Optional consistency check between owner and student_id upon collect_ by **[nthiery](https://github.com/nthiery)**
* [(#1110)](https://github.com/jupyter/nbgrader/pull/1110): _Systematic use of utils.get_username instead of $USER_ by **[nthiery](https://github.com/nthiery)**
* [(#1109)](https://github.com/jupyter/nbgrader/pull/1109): _naming the temporary directories in tests_ by **[danielmaitre](https://github.com/danielmaitre)**
* [(#1108)](https://github.com/jupyter/nbgrader/pull/1108): _Extended support for filtering files copied in the exchange_ by **[nthiery](https://github.com/nthiery)**
* [(#1106)](https://github.com/jupyter/nbgrader/pull/1106): _Remove testing of python 3.4_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1105)](https://github.com/jupyter/nbgrader/pull/1105): _Remove extra keys in nbgrader metadata and better schema mismatch errors_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1102)](https://github.com/jupyter/nbgrader/pull/1102): _Only build docs with one version of python_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1101)](https://github.com/jupyter/nbgrader/pull/1101): _Add jupyter education book to third party resources_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1100)](https://github.com/jupyter/nbgrader/pull/1100): _Run test in the `python` group in parallel using pytest-xdist_ by **[BertR](https://github.com/BertR)**
* [(#1099)](https://github.com/jupyter/nbgrader/pull/1099): _Add course table, add course_id column to assignment_ by **[BertR](https://github.com/BertR)**
* [(#1098)](https://github.com/jupyter/nbgrader/pull/1098): _Customizable student ID in `nbgrader submit`_ by **[nthiery](https://github.com/nthiery)**
* [(#1094)](https://github.com/jupyter/nbgrader/pull/1094): _Update license_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1093)](https://github.com/jupyter/nbgrader/pull/1093): _Add authentication plugin support_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1088)](https://github.com/jupyter/nbgrader/pull/1088): _Remove version requirement from urllib3_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1071)](https://github.com/jupyter/nbgrader/pull/1071): _student and assignment selection in exportapp implemented_ by **[ddland](https://github.com/ddland)**
* [(#1040)](https://github.com/jupyter/nbgrader/pull/1040): _Restrict access for students to different courses_ by **[sigurdurb](https://github.com/sigurdurb)**
* [(#1036)](https://github.com/jupyter/nbgrader/pull/1036): _Add a general lms user id column to the student table_ by **[sigurdurb](https://github.com/sigurdurb)**
* [(#1001)](https://github.com/jupyter/nbgrader/pull/1001): _Allow setting a different assignment dir for students than the root notebook directory_ by **[rkdarst](https://github.com/rkdarst)**

##### [jupyter/nbviewer](https://github.com/jupyter/nbviewer)
* [(#833)](https://github.com/jupyter/nbviewer/pull/833): _(1) Fixed FAQ page, (2) made front page compatible with JupyterHub_ by **[krinsman](https://github.com/krinsman)**

##### [jupyter/notebook](https://github.com/jupyter/notebook)
* [(#4661)](https://github.com/jupyter/notebook/pull/4661): _Add id for command palette button group_ by **[tmetzl](https://github.com/tmetzl)**
* [(#4654)](https://github.com/jupyter/notebook/pull/4654): _.travis.yml: The 'sudo' tag is now deprecated in Travis CI_ by **[abhinavsagar](https://github.com/abhinavsagar)**
* [(#4651)](https://github.com/jupyter/notebook/pull/4651): _Fix French translation typo_ by **[tholoz](https://github.com/tholoz)**
* [(#4642)](https://github.com/jupyter/notebook/pull/4642): _Use JavaScript to redirect users_ by **[betatim](https://github.com/betatim)**
* [(#4591)](https://github.com/jupyter/notebook/pull/4591): _Update Working With Markdown Cells.ipynb_ by **[dalanicolai](https://github.com/dalanicolai)**

##### [jupyter/qtconsole](https://github.com/jupyter/qtconsole)
* [(#344)](https://github.com/jupyter/qtconsole/pull/344): _Several improvements to the Readme_ by **[ccordoba12](https://github.com/ccordoba12)**
* [(#343)](https://github.com/jupyter/qtconsole/pull/343): _Move setup.py to use setuptools instead of distutils_ by **[ccordoba12](https://github.com/ccordoba12)**
* [(#342)](https://github.com/jupyter/qtconsole/pull/342): _Update Changelog with 4.5 changes_ by **[ccordoba12](https://github.com/ccordoba12)**
* [(#341)](https://github.com/jupyter/qtconsole/pull/341): _PR: Use new traitlets API with decorators_ by **[dalthviz](https://github.com/dalthviz)**
* [(#340)](https://github.com/jupyter/qtconsole/pull/340): _Remove metadata entry from setup.cfg_ by **[ccordoba12](https://github.com/ccordoba12)**
* [(#339)](https://github.com/jupyter/qtconsole/pull/339): _Use Readme.md for long_description in setup.py_ by **[ccordoba12](https://github.com/ccordoba12)**
* [(#338)](https://github.com/jupyter/qtconsole/pull/338): _PR: Add Comms to qtconsole_ by **[impact27](https://github.com/impact27)**
* [(#336)](https://github.com/jupyter/qtconsole/pull/336): _PR: Add language name as attribute of the JupyterWidget_ by **[dalthviz](https://github.com/dalthviz)**

##### [jupyter/repo2docker](https://github.com/jupyter/repo2docker)
* [(#705)](https://github.com/jupyter/repo2docker/pull/705): _base docker image on alpine_ by **[minrk](https://github.com/minrk)**
* [(#703)](https://github.com/jupyter/repo2docker/pull/703): _add `git-lfs` to dockerfile used for the repo2docker image_ by **[minrk](https://github.com/minrk)**
* [(#699)](https://github.com/jupyter/repo2docker/pull/699): _[MRG] Add auto-formatting setup_ by **[betatim](https://github.com/betatim)**
* [(#697)](https://github.com/jupyter/repo2docker/pull/697): _Upgrade JupyterHub to 1.0_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#695)](https://github.com/jupyter/repo2docker/pull/695): _[MRG] Update verification of Node install_ by **[betatim](https://github.com/betatim)**
* [(#693)](https://github.com/jupyter/repo2docker/pull/693): _[MRG] Zenodo content provider_ by **[betatim](https://github.com/betatim)**
* [(#692)](https://github.com/jupyter/repo2docker/pull/692): _Adding contentprovider documentation_ by **[choldgraf](https://github.com/choldgraf)**
* [(#690)](https://github.com/jupyter/repo2docker/pull/690): _set CONDA_DEFAULT_ENV_ by **[minrk](https://github.com/minrk)**
* [(#689)](https://github.com/jupyter/repo2docker/pull/689): _Switch Travis CI to Ubuntu Xenial 16.04_ by **[jrbourbeau](https://github.com/jrbourbeau)**
* [(#684)](https://github.com/jupyter/repo2docker/pull/684): _[MRG] Drop support for Python 3.4_ by **[betatim](https://github.com/betatim)**
* [(#683)](https://github.com/jupyter/repo2docker/pull/683): _[MRG] Use getpass instead of pwd to fetch username_ by **[betatim](https://github.com/betatim)**

##### [jupyter/telemetry](https://github.com/jupyter/telemetry)
* [(#4)](https://github.com/jupyter/telemetry/pull/4): _Address open questions regarding event schemas_ by **[jaipreet-s](https://github.com/jaipreet-s)**
* [(#1)](https://github.com/jupyter/telemetry/pull/1): _Jupyter Enhancement Proposal(JEP) document_ by **[jaipreet-s](https://github.com/jaipreet-s)**


#### jupyter-widgets

##### [jupyter-widgets/ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet)
* [(#384)](https://github.com/jupyter-widgets/ipyleaflet/pull/384): _Add Usage section in the docs_ by **[martinRenou](https://github.com/martinRenou)**
* [(#383)](https://github.com/jupyter-widgets/ipyleaflet/pull/383): _Add jlab themes support_ by **[martinRenou](https://github.com/martinRenou)**
* [(#382)](https://github.com/jupyter-widgets/ipyleaflet/pull/382): _Add min_native_zoom and max_native_zoom for TileLayer_ by **[martinRenou](https://github.com/martinRenou)**
* [(#381)](https://github.com/jupyter-widgets/ipyleaflet/pull/381): _Add Select GeoJSON example_ by **[martinRenou](https://github.com/martinRenou)**
* [(#378)](https://github.com/jupyter-widgets/ipyleaflet/pull/378): _Add CustomTileServer example_ by **[deeplook](https://github.com/deeplook)**
* [(#376)](https://github.com/jupyter-widgets/ipyleaflet/pull/376): _Fix wrong requirement string_ by **[Juanlu001](https://github.com/Juanlu001)**
* [(#368)](https://github.com/jupyter-widgets/ipyleaflet/pull/368): _geodata class_ by **[VasavanThiru](https://github.com/VasavanThiru)**
* [(#366)](https://github.com/jupyter-widgets/ipyleaflet/pull/366): _Adding an example of selecting GeoJSON features by clicking on the leaflet layer_ by **[mpdwulit](https://github.com/mpdwulit)**

##### [jupyter-widgets/ipywidgets](https://github.com/jupyter-widgets/ipywidgets)
* [(#2447)](https://github.com/jupyter-widgets/ipywidgets/pull/2447): _Update documentation from the widgets tutorial_ by **[mwcraig](https://github.com/mwcraig)**
* [(#2442)](https://github.com/jupyter-widgets/ipywidgets/pull/2442): _Fix spelling error_ by **[edolley](https://github.com/edolley)**
* [(#2438)](https://github.com/jupyter-widgets/ipywidgets/pull/2438): _Update to latest JupyterLab_ by **[SylvainCorlay](https://github.com/SylvainCorlay)**
* [(#2435)](https://github.com/jupyter-widgets/ipywidgets/pull/2435): _FloatRangeSlider `value` logic_ by **[lheagy](https://github.com/lheagy)**
* [(#2429)](https://github.com/jupyter-widgets/ipywidgets/pull/2429): _Doc: add nodejs install command_ by **[martinRenou](https://github.com/martinRenou)**
* [(#2425)](https://github.com/jupyter-widgets/ipywidgets/pull/2425): _Remove unused scrollIfNeeded and StyleModel import_ by **[jtpio](https://github.com/jtpio)**
* [(#2424)](https://github.com/jupyter-widgets/ipywidgets/pull/2424): _update the mention of jupyter_sphinx_ by **[akhmerov](https://github.com/akhmerov)**
* [(#2423)](https://github.com/jupyter-widgets/ipywidgets/pull/2423): _#2422 provide type explicitly to fix complilation_ by **[throwable-one](https://github.com/throwable-one)**
* [(#2421)](https://github.com/jupyter-widgets/ipywidgets/pull/2421): _Update for JLab 1.0a5._ by **[jasongrout](https://github.com/jasongrout)**
* [(#2418)](https://github.com/jupyter-widgets/ipywidgets/pull/2418): _Drop postcss_ by **[SylvainCorlay](https://github.com/SylvainCorlay)**
* [(#2409)](https://github.com/jupyter-widgets/ipywidgets/pull/2409): _Remove the overflow exception for the shorthand property_ by **[SylvainCorlay](https://github.com/SylvainCorlay)**
* [(#2315)](https://github.com/jupyter-widgets/ipywidgets/pull/2315): _Enable noImplicitAny compiler check, fix tslint errors._ by **[blois](https://github.com/blois)**
* [(#2303)](https://github.com/jupyter-widgets/ipywidgets/pull/2303): _Fix DeprecationWarning in Python 3.7_ by **[Juanlu001](https://github.com/Juanlu001)**

##### [jupyter-widgets/jupyter-sphinx](https://github.com/jupyter-widgets/jupyter-sphinx)
* [(#51)](https://github.com/jupyter-widgets/jupyter-sphinx/pull/51): _mention that Jupyter Sphinx works with autodoc_ by **[jbweston](https://github.com/jbweston)**
* [(#49)](https://github.com/jupyter-widgets/jupyter-sphinx/pull/49): _add javascript support_ by **[basnijholt](https://github.com/basnijholt)**
* [(#46)](https://github.com/jupyter-widgets/jupyter-sphinx/pull/46): _configure travis_ by **[akhmerov](https://github.com/akhmerov)**
* [(#44)](https://github.com/jupyter-widgets/jupyter-sphinx/pull/44): _remove embed_widgets extension_ by **[jbweston](https://github.com/jbweston)**
* [(#24)](https://github.com/jupyter-widgets/jupyter-sphinx/pull/24): _add support for rendering ipywidgets_ by **[jbweston](https://github.com/jbweston)**

##### [jupyter-widgets/pythreejs](https://github.com/jupyter-widgets/pythreejs)
* [(#278)](https://github.com/jupyter-widgets/pythreejs/pull/278): _Update deps within semver range_ by **[vidartf](https://github.com/vidartf)**
* [(#277)](https://github.com/jupyter-widgets/pythreejs/pull/277): _Fix computeBoundingSphere_ by **[vidartf](https://github.com/vidartf)**
* [(#272)](https://github.com/jupyter-widgets/pythreejs/pull/272): _Update traits.py for collections deprecation warning_ by **[GenevieveBuckley](https://github.com/GenevieveBuckley)**
* [(#271)](https://github.com/jupyter-widgets/pythreejs/pull/271): _Update pythreejs.py to silence traitlets warning_ by **[GenevieveBuckley](https://github.com/GenevieveBuckley)**

##### [jupyter-widgets/tutorial](https://github.com/jupyter-widgets/tutorial)
* [(#45)](https://github.com/jupyter-widgets/tutorial/pull/45): _Address overflow/overflow_x since users will be on mix of 7.4 and 7.5_ by **[mwcraig](https://github.com/mwcraig)**
* [(#44)](https://github.com/jupyter-widgets/tutorial/pull/44): _Condense install instructions a bit more and update check script_ by **[mwcraig](https://github.com/mwcraig)**
* [(#43)](https://github.com/jupyter-widgets/tutorial/pull/43): _Add event example with ipytree_ by **[martinRenou](https://github.com/martinRenou)**
* [(#41)](https://github.com/jupyter-widgets/tutorial/pull/41): _Improve Notebook introductions_ by **[martinRenou](https://github.com/martinRenou)**
* [(#39)](https://github.com/jupyter-widgets/tutorial/pull/39): _Update introduction_ by **[martinRenou](https://github.com/martinRenou)**
* [(#38)](https://github.com/jupyter-widgets/tutorial/pull/38): _Merge install commands_ by **[martinRenou](https://github.com/martinRenou)**
* [(#37)](https://github.com/jupyter-widgets/tutorial/pull/37): _Fix dead link in 03.00-Widget_Basics.ipynb_ by **[kohr-h](https://github.com/kohr-h)**
* [(#35)](https://github.com/jupyter-widgets/tutorial/pull/35): _Add ipympl example Notebook_ by **[martinRenou](https://github.com/martinRenou)**
* [(#34)](https://github.com/jupyter-widgets/tutorial/pull/34): _Add ipysheet example_ by **[martinRenou](https://github.com/martinRenou)**
* [(#33)](https://github.com/jupyter-widgets/tutorial/pull/33): _Add vuetify/voila example_ by **[maartenbreddels](https://github.com/maartenbreddels)**
* [(#32)](https://github.com/jupyter-widgets/tutorial/pull/32): _Pull in changes from the ipywidgets documentation _ by **[mwcraig](https://github.com/mwcraig)**
* [(#31)](https://github.com/jupyter-widgets/tutorial/pull/31): _Add ipytree example_ by **[martinRenou](https://github.com/martinRenou)**
* [(#30)](https://github.com/jupyter-widgets/tutorial/pull/30): _Update dependencies_ by **[martinRenou](https://github.com/martinRenou)**
* [(#29)](https://github.com/jupyter-widgets/tutorial/pull/29): _Add WidgetControl example on the ipyleaflet notebook_ by **[martinRenou](https://github.com/martinRenou)**
* [(#28)](https://github.com/jupyter-widgets/tutorial/pull/28): _Add some disclaimers about the current state of installation instructions_ by **[mwcraig](https://github.com/mwcraig)**
* [(#26)](https://github.com/jupyter-widgets/tutorial/pull/26): _Update ipyleaflet notebook_ by **[martinRenou](https://github.com/martinRenou)**

##### [jupyter-widgets/widget-cookiecutter](https://github.com/jupyter-widgets/widget-cookiecutter)
* [(#45)](https://github.com/jupyter-widgets/widget-cookiecutter/pull/45): _add watch command, normalize plugin name, fix hello world default_ by **[wolfv](https://github.com/wolfv)**


#### jupyterhub

##### [jupyterhub/alabaster-jupyterhub](https://github.com/jupyterhub/alabaster-jupyterhub)
* [(#24)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/24): _dev version add_ by **[choldgraf](https://github.com/choldgraf)**
* [(#23)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/23): _bump minor version_ by **[choldgraf](https://github.com/choldgraf)**
* [(#22)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/22): _setting footer max-width to 650px_ by **[choldgraf](https://github.com/choldgraf)**
* [(#21)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/21): _dev version bump_ by **[choldgraf](https://github.com/choldgraf)**
* [(#20)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/20): _bump version for minor release_ by **[choldgraf](https://github.com/choldgraf)**
* [(#19)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/19): _small fixes to whitespace_ by **[choldgraf](https://github.com/choldgraf)**
* [(#18)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/18): _dev version bump 1.7_ by **[choldgraf](https://github.com/choldgraf)**
* [(#17)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/17): _version bump_ by **[choldgraf](https://github.com/choldgraf)**
* [(#16)](https://github.com/jupyterhub/alabaster-jupyterhub/pull/16): _enabling right sidebar scrolling and spacing tweaks and edit on github_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyterhub/batchspawner](https://github.com/jupyterhub/batchspawner)
* [(#145)](https://github.com/jupyterhub/batchspawner/pull/145): _Add note on spawner maintainers_ by **[rkdarst](https://github.com/rkdarst)**
* [(#143)](https://github.com/jupyterhub/batchspawner/pull/143): _Singleuserapp and 1.0_ by **[rkdarst](https://github.com/rkdarst)**
* [(#142)](https://github.com/jupyterhub/batchspawner/pull/142): _Drop backwards compatibility to jupyterhub==0.9.6 and python==3.5_ by **[rkdarst](https://github.com/rkdarst)**
* [(#141)](https://github.com/jupyterhub/batchspawner/pull/141): _Allow batchspawner to be used with different notebook singleuser cmd_ by **[cmd-ntrf](https://github.com/cmd-ntrf)**
* [(#140)](https://github.com/jupyterhub/batchspawner/pull/140): _Fix typos_ by **[kinow](https://github.com/kinow)**
* [(#139)](https://github.com/jupyterhub/batchspawner/pull/139): _Replace current_ip and current_port by ip and port_ by **[cmd-ntrf](https://github.com/cmd-ntrf)**
* [(#116)](https://github.com/jupyterhub/batchspawner/pull/116): _Update req_srun help string_ by **[rkdarst](https://github.com/rkdarst)**

##### [jupyterhub/binderhub](https://github.com/jupyterhub/binderhub)
* [(#881)](https://github.com/jupyterhub/binderhub/pull/881): _[MRG] Add basic news header to the HTML templates_ by **[betatim](https://github.com/betatim)**
* [(#879)](https://github.com/jupyterhub/binderhub/pull/879): _[MRG] Manually list BinderHub dependencies in docs requirements_ by **[betatim](https://github.com/betatim)**
* [(#878)](https://github.com/jupyterhub/binderhub/pull/878): _[MRG] Add instructions for setting up an Azure Container Registry_ by **[sgibson91](https://github.com/sgibson91)**
* [(#876)](https://github.com/jupyterhub/binderhub/pull/876): _document launch schema version 3_ by **[bitnik](https://github.com/bitnik)**
* [(#873)](https://github.com/jupyterhub/binderhub/pull/873): _[MRG] Fix typo in OVH link_ by **[betatim](https://github.com/betatim)**
* [(#872)](https://github.com/jupyterhub/binderhub/pull/872): _Add 'origin' field to launch events_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#870)](https://github.com/jupyterhub/binderhub/pull/870): _[MRG] Add Zenodo provider_ by **[betatim](https://github.com/betatim)**
* [(#867)](https://github.com/jupyterhub/binderhub/pull/867): _[MRG] Add extraEnv to the BinderHub deployment_ by **[betatim](https://github.com/betatim)**
* [(#866)](https://github.com/jupyterhub/binderhub/pull/866): _[MRG] Add OVH BinderHub to list of known deployments_ by **[betatim](https://github.com/betatim)**
* [(#865)](https://github.com/jupyterhub/binderhub/pull/865): _[MRG] Make base badge URL available in the loading screen as well_ by **[betatim](https://github.com/betatim)**
* [(#860)](https://github.com/jupyterhub/binderhub/pull/860): _[MRG] rename variable and restructure API docs_ by **[nuest](https://github.com/nuest)**
* [(#859)](https://github.com/jupyterhub/binderhub/pull/859): _[MRG] Make the URL used to generate launch badges configurable_ by **[betatim](https://github.com/betatim)**
* [(#858)](https://github.com/jupyterhub/binderhub/pull/858): _[MRG] Tweak loading messages_ by **[betatim](https://github.com/betatim)**
* [(#857)](https://github.com/jupyterhub/binderhub/pull/857): _fix tolerations in dind daemonset and add tolerations to image-cleaner_ by **[jhamman](https://github.com/jhamman)**
* [(#856)](https://github.com/jupyterhub/binderhub/pull/856): _add user tolerations to dind daemonset_ by **[jhamman](https://github.com/jhamman)**
* [(#853)](https://github.com/jupyterhub/binderhub/pull/853): _add default toleration to build pods_ by **[jhamman](https://github.com/jhamman)**
* [(#784)](https://github.com/jupyterhub/binderhub/pull/784): _[MRG] use chartpress 0.3_ by **[minrk](https://github.com/minrk)**

##### [jupyterhub/chartpress](https://github.com/jupyterhub/chartpress)
* [(#36)](https://github.com/jupyterhub/chartpress/pull/36): _Add initial setup when starting from scratch_ by **[manics](https://github.com/manics)**
* [(#34)](https://github.com/jupyterhub/chartpress/pull/34): _avoid mangling of quotes in rendered charts (#1)_ by **[rokroskar](https://github.com/rokroskar)**

##### [jupyterhub/jupyter-server-proxy](https://github.com/jupyterhub/jupyter-server-proxy)
* [(#132)](https://github.com/jupyterhub/jupyter-server-proxy/pull/132): _install-doc-update_ by **[quasiben](https://github.com/quasiben)**

##### [jupyterhub/jupyterhub](https://github.com/jupyterhub/jupyterhub)
* [(#2609)](https://github.com/jupyterhub/jupyterhub/pull/2609): _Replace existing redirect code by Tornado's addslash decorator_ by **[kinow](https://github.com/kinow)**
* [(#2605)](https://github.com/jupyterhub/jupyterhub/pull/2605): _add activity_resolution config_ by **[minrk](https://github.com/minrk)**
* [(#2590)](https://github.com/jupyterhub/jupyterhub/pull/2590): _Change API description to be a valid OpenAPI spec_ by **[sbrunk](https://github.com/sbrunk)**
* [(#2588)](https://github.com/jupyterhub/jupyterhub/pull/2588): _Define default values for HubAuth ssl traitlets_ by **[cmd-ntrf](https://github.com/cmd-ntrf)**
* [(#2584)](https://github.com/jupyterhub/jupyterhub/pull/2584): _re-raise exceptions in launch_instance_async_ by **[minrk](https://github.com/minrk)**
* [(#2577)](https://github.com/jupyterhub/jupyterhub/pull/2577): _Hide Stop My Server red button after server stopped._ by **[aar0nTw](https://github.com/aar0nTw)**
* [(#2570)](https://github.com/jupyterhub/jupyterhub/pull/2570): _Make announcements visible without custom HTML_ by **[consideRatio](https://github.com/consideRatio)**
* [(#2565)](https://github.com/jupyterhub/jupyterhub/pull/2565): _Update link of `changelog`_ by **[iblis17](https://github.com/iblis17)**
* [(#2560)](https://github.com/jupyterhub/jupyterhub/pull/2560): _Jupyterhub: use previous exit strategy for Windows_ by **[adelcast](https://github.com/adelcast)**

##### [jupyterhub/mybinder.org-deploy](https://github.com/jupyterhub/mybinder.org-deploy)
* [(#1039)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1039): _Make the federation redirector smarter_ by **[betatim](https://github.com/betatim)**
* [(#1037)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1037): _Send 15% to OVH_ by **[betatim](https://github.com/betatim)**
* [(#1035)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1035): _New GitHub credentials for OVH_ by **[betatim](https://github.com/betatim)**
* [(#1034)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1034): _Reuse the GKE GitHub credentials for OVH_ by **[betatim](https://github.com/betatim)**
* [(#1033)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1033): _Reduce OVH fraction to zero_ by **[betatim](https://github.com/betatim)**
* [(#1032)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1032): _Reduce OVH fractionn while we fix the GitHub API issue_ by **[betatim](https://github.com/betatim)**
* [(#1030)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1030): _Add a banner message for the survey_ by **[betatim](https://github.com/betatim)**
* [(#1029)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1029): _binderhub: 908c443...c8ab0a3_ by **[henchbot](https://github.com/henchbot)**
* [(#1028)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1028): _repo2docker: 0693fc09...9f110584_ by **[henchbot](https://github.com/henchbot)**
* [(#1027)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1027): _repo2docker: f19e159d...0693fc09_ by **[henchbot](https://github.com/henchbot)**
* [(#1026)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1026): _Increase OVH to 25%_ by **[betatim](https://github.com/betatim)**
* [(#1025)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1025): _binderhub: 4300f2c...908c443_ by **[henchbot](https://github.com/henchbot)**
* [(#1024)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1024): _Increase OVH traffic share to 20%_ by **[betatim](https://github.com/betatim)**
* [(#1022)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1022): _binderhub: 3dd5511...4300f2c_ by **[henchbot](https://github.com/henchbot)**
* [(#1020)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1020): _Lower traffic fraction sent to OVH_ by **[betatim](https://github.com/betatim)**
* [(#1019)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1019): _Make OVH ingress config similar to gke one_ by **[betatim](https://github.com/betatim)**
* [(#1018)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1018): _Change OVH ingress to serve two hostnames_ by **[betatim](https://github.com/betatim)**
* [(#1017)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1017): _Increase memory request and limit for nginx-ingress_ by **[betatim](https://github.com/betatim)**
* [(#1015)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1015): _Update resource limits on OVH_ by **[betatim](https://github.com/betatim)**
* [(#1014)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1014): _Send 10% of the traffic to OVH_ by **[betatim](https://github.com/betatim)**
* [(#1013)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1013): _Increase OVH cluster to 5% of traffic_ by **[betatim](https://github.com/betatim)**
* [(#1012)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1012): _Create placeholder for events archiver service account_ by **[betatim](https://github.com/betatim)**
* [(#1011)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1011): _[WIP] Add secret and setup to write events from OVH_ by **[betatim](https://github.com/betatim)**
* [(#1010)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1010): _Automatically deploy ovh ingress for the 'ovh.mybinder.org' certificate_ by **[mael-le-gal](https://github.com/mael-le-gal)**
* [(#1009)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1009): _Setup headers for CORS_ by **[betatim](https://github.com/betatim)**
* [(#1008)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1008): _Handle badges and about page in the redirector_ by **[betatim](https://github.com/betatim)**
* [(#1007)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1007): _Enable federation for mybinder.org_ by **[betatim](https://github.com/betatim)**
* [(#1006)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1006): _Revert "Disable redirector on staging" to enable it again_ by **[betatim](https://github.com/betatim)**
* [(#1005)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1005): _Disable redirector on staging_ by **[betatim](https://github.com/betatim)**
* [(#1004)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1004): _Switch to tornado logging and handle exceptions_ by **[betatim](https://github.com/betatim)**
* [(#1003)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1003): _Add a second hostname for the staging cluster_ by **[betatim](https://github.com/betatim)**
* [(#1002)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1002): _Set badge base URL for all clusters_ by **[betatim](https://github.com/betatim)**
* [(#1001)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1001): _Fix staging federation_ by **[betatim](https://github.com/betatim)**
* [(#1000)](https://github.com/jupyterhub/mybinder.org-deploy/pull/1000): _Enable frederick on staging, change staging hostnames around_ by **[betatim](https://github.com/betatim)**
* [(#997)](https://github.com/jupyterhub/mybinder.org-deploy/pull/997): _repo2docker: 80fbc639...f19e159d_ by **[henchbot](https://github.com/henchbot)**
* [(#996)](https://github.com/jupyterhub/mybinder.org-deploy/pull/996): _binderhub: 851025f...9469b37_ by **[henchbot](https://github.com/henchbot)**
* [(#995)](https://github.com/jupyterhub/mybinder.org-deploy/pull/995): _Revert "binderhub: 851025f...0d300b8"_ by **[betatim](https://github.com/betatim)**
* [(#994)](https://github.com/jupyterhub/mybinder.org-deploy/pull/994): _binderhub: 851025f...0d300b8_ by **[henchbot](https://github.com/henchbot)**
* [(#993)](https://github.com/jupyterhub/mybinder.org-deploy/pull/993): _binderhub: 9f83710...851025f_ by **[henchbot](https://github.com/henchbot)**
* [(#992)](https://github.com/jupyterhub/mybinder.org-deploy/pull/992): _Use "weird" timeouts_ by **[betatim](https://github.com/betatim)**
* [(#991)](https://github.com/jupyterhub/mybinder.org-deploy/pull/991): _repo2docker: 51898274...80fbc639_ by **[henchbot](https://github.com/henchbot)**
* [(#990)](https://github.com/jupyterhub/mybinder.org-deploy/pull/990): _repo2docker: 80b979f8...51898274_ by **[henchbot](https://github.com/henchbot)**
* [(#988)](https://github.com/jupyterhub/mybinder.org-deploy/pull/988): _Add README for federation redirector_ by **[betatim](https://github.com/betatim)**
* [(#987)](https://github.com/jupyterhub/mybinder.org-deploy/pull/987): _Add a redirector for the mybinder.org federation_ by **[betatim](https://github.com/betatim)**
* [(#984)](https://github.com/jupyterhub/mybinder.org-deploy/pull/984): _Add hmharshit/mltraining which is a fork of a previously banned repo_ by **[betatim](https://github.com/betatim)**
* [(#983)](https://github.com/jupyterhub/mybinder.org-deploy/pull/983): _Bump repo2docker and binderhub_ by **[betatim](https://github.com/betatim)**
* [(#982)](https://github.com/jupyterhub/mybinder.org-deploy/pull/982): _Revert "repo2docker: 91309cab...80b979f8"_ by **[choldgraf](https://github.com/choldgraf)**
* [(#981)](https://github.com/jupyterhub/mybinder.org-deploy/pull/981): _Revert "binderhub: 82fc209...9f83710"_ by **[henchc](https://github.com/henchc)**
* [(#979)](https://github.com/jupyterhub/mybinder.org-deploy/pull/979): _binderhub: 82fc209...9f83710_ by **[henchbot](https://github.com/henchbot)**
* [(#978)](https://github.com/jupyterhub/mybinder.org-deploy/pull/978): _repo2docker: 91309cab...80b979f8_ by **[henchbot](https://github.com/henchbot)**
* [(#976)](https://github.com/jupyterhub/mybinder.org-deploy/pull/976): _binderhub: 10ac4d8...82fc209_ by **[henchbot](https://github.com/henchbot)**
* [(#975)](https://github.com/jupyterhub/mybinder.org-deploy/pull/975): _Fix typo in repo ban_ by **[betatim](https://github.com/betatim)**
* [(#974)](https://github.com/jupyterhub/mybinder.org-deploy/pull/974): _Ban shishirchoudharygic/mltraining_ by **[betatim](https://github.com/betatim)**
* [(#973)](https://github.com/jupyterhub/mybinder.org-deploy/pull/973): _Open an additional port for the xroot protocol_ by **[betatim](https://github.com/betatim)**
* [(#972)](https://github.com/jupyterhub/mybinder.org-deploy/pull/972): _repo2docker: 25ebca04...91309cab_ by **[henchbot](https://github.com/henchbot)**
* [(#971)](https://github.com/jupyterhub/mybinder.org-deploy/pull/971): _Ban hmharshit/cn-ait for the moment_ by **[betatim](https://github.com/betatim)**
* [(#970)](https://github.com/jupyterhub/mybinder.org-deploy/pull/970): _Adding new configuration for OVH cluster_ by **[mael-le-gal](https://github.com/mael-le-gal)**

##### [jupyterhub/nbgitpuller](https://github.com/jupyterhub/nbgitpuller)
* [(#97)](https://github.com/jupyterhub/nbgitpuller/pull/97): _Allow git@example.com:repo links_ by **[cristiklein](https://github.com/cristiklein)**
* [(#92)](https://github.com/jupyterhub/nbgitpuller/pull/92): _Redo documentation_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#88)](https://github.com/jupyterhub/nbgitpuller/pull/88): _documentation clarification_ by **[parmentelat](https://github.com/parmentelat)**
* [(#82)](https://github.com/jupyterhub/nbgitpuller/pull/82): _circle config to push docs_ by **[choldgraf](https://github.com/choldgraf)**
* [(#81)](https://github.com/jupyterhub/nbgitpuller/pull/81): _adding documentation_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyterhub/oauthenticator](https://github.com/jupyterhub/oauthenticator)
* [(#261)](https://github.com/jupyterhub/oauthenticator/pull/261): _Allow option to disable basic auth header in generic oauth_ by **[sunnielyu](https://github.com/sunnielyu)**

##### [jupyterhub/systemdspawner](https://github.com/jupyterhub/systemdspawner)
* [(#53)](https://github.com/jupyterhub/systemdspawner/pull/53): _fix for Centos 7_ by **[mortbauer](https://github.com/mortbauer)**

##### [jupyterhub/team-compass](https://github.com/jupyterhub/team-compass)
* [(#173)](https://github.com/jupyterhub/team-compass/pull/173): _fixing link in onboarding docs_ by **[choldgraf](https://github.com/choldgraf)**
* [(#167)](https://github.com/jupyterhub/team-compass/pull/167): _adding onboarding docs_ by **[choldgraf](https://github.com/choldgraf)**
* [(#166)](https://github.com/jupyterhub/team-compass/pull/166): _Adding Sarah to the blue team_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyterhub/the-littlest-jupyterhub](https://github.com/jupyterhub/the-littlest-jupyterhub)
* [(#380)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/380): _Tell apt-get to never ask questions_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#376)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/376): _Typo fix: `s` -> `is`_ by **[jtpio](https://github.com/jtpio)**
* [(#374)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/374): _Fix typo: missing "c" for instance_ by **[jtpio](https://github.com/jtpio)**
* [(#372)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/372): _Minor typo fix: praticular -> particular_ by **[jtpio](https://github.com/jtpio)**
* [(#371)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/371): _Add Tutorial for OVH_ by **[jtpio](https://github.com/jtpio)**
* [(#370)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/370): _Clarify the steps to build the docs locally_ by **[jtpio](https://github.com/jtpio)**
* [(#367)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/367): _Fix typo in README link_ by **[pbugnion](https://github.com/pbugnion)**
* [(#366)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/366): _Add idle culler_ by **[GeorgianaElena](https://github.com/GeorgianaElena)**
* [(#365)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/365): _Add tmpauthenticator by default to TLJH_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#363)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/363): _Fix typo: cohnfig -> config_ by **[staeiou](https://github.com/staeiou)**
* [(#360)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/360): _Add custom hub package & config hooks_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#359)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/359): _Install & use pycurl for requests_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#358)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/358): _Minor azure doc cleanup_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#357)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/357): _Suppress insecure HTTPS warning when upgrading TLJH_ by **[GeorgianaElena](https://github.com/GeorgianaElena)**
* [(#355)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/355): _Fixed out of date config directory listed in docs for tljh-config_ by **[JuanCab](https://github.com/JuanCab)**
* [(#352)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/352): _Add "tljh-config unset" option_ by **[GeorgianaElena](https://github.com/GeorgianaElena)**
* [(#347)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/347): _Upgrade while https enabled_ by **[GeorgianaElena](https://github.com/GeorgianaElena)**
* [(#342)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/342): _Add instructions to deploy on Azure_ by **[trallard](https://github.com/trallard)**
* [(#339)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/339): _Retry downloading traefik if it fails_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#337)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/337): _Provide much better error messages_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#335)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/335): _Limit memory available in integration tests_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#331)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/331): _Remove stray = in authenticator configuration example_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#329)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/329): _Minor cleanup of custom server install documents_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#328)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/328): _Cleanup HTTPS documentation_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#327)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/327): _Add note about not running on your own laptop or in Docker_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#313)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/313): _Upgrade to JupyterHub 1.0_ by **[minrk](https://github.com/minrk)**
* [(#242)](https://github.com/jupyterhub/the-littlest-jupyterhub/pull/242): _Adds the universe repository to the used sources_ by **[owah](https://github.com/owah)**

##### [jupyterhub/traefik-proxy](https://github.com/jupyterhub/traefik-proxy)
* [(#57)](https://github.com/jupyterhub/traefik-proxy/pull/57): _TraefikConsulProxy_ by **[GeorgianaElena](https://github.com/GeorgianaElena)**

##### [jupyterhub/wrapspawner](https://github.com/jupyterhub/wrapspawner)
* [(#28)](https://github.com/jupyterhub/wrapspawner/pull/28): _Update config instructions and profiles parameters_ by **[cmd-ntrf](https://github.com/cmd-ntrf)**
* [(#27)](https://github.com/jupyterhub/wrapspawner/pull/27): _Create link between WrapSpawner and its child spawner traitlets_ by **[cmd-ntrf](https://github.com/cmd-ntrf)**

##### [jupyterhub/zero-to-jupyterhub-k8s](https://github.com/jupyterhub/zero-to-jupyterhub-k8s)
* [(#1309)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1309): _add explanation for lifecycle_hooks in kubespawner_override_ by **[clancychilds](https://github.com/clancychilds)**
* [(#1304)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1304): _Update chart version to 0.8.2 in the docs_ by **[jtpio](https://github.com/jtpio)**
* [(#1299)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1299): _tmpauthenticator 0.6 needed for jupyterhub 1.0_ by **[manics](https://github.com/manics)**
* [(#1298)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1298): _ Fix azure cli VMSSPreview feature register command _ by **[dazzag24](https://github.com/dazzag24)**
* [(#1294)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1294): _Unbreak git build_ by **[joshbode](https://github.com/joshbode)**
* [(#1291)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1291): _Update Dockerfile to JH 1.0_ by **[vilhelmen](https://github.com/vilhelmen)**
* [(#1289)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1289): _Suggest quotes around tag._ by **[danielballan](https://github.com/danielballan)**
* [(#1288)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1288): _Include jupyter-firstuseauthenticator._ by **[danielballan](https://github.com/danielballan)**
* [(#1272)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/pull/1272): _enable user scheduler by default_ by **[minrk](https://github.com/minrk)**


#### jupyterlab

##### [jupyterlab/jupyter-renderers](https://github.com/jupyterlab/jupyter-renderers)
* [(#193)](https://github.com/jupyterlab/jupyter-renderers/pull/193): _Add extension number for Vega package_ by **[domoritz](https://github.com/domoritz)**
* [(#184)](https://github.com/jupyterlab/jupyter-renderers/pull/184): _Add Vega 4 extension_ by **[domoritz](https://github.com/domoritz)**

##### [jupyterlab/jupyterlab](https://github.com/jupyterlab/jupyterlab)
* [(#6661)](https://github.com/jupyterlab/jupyterlab/pull/6661): _Fix typo in extension_dev document._ by **[danielballan](https://github.com/danielballan)**
* [(#6653)](https://github.com/jupyterlab/jupyterlab/pull/6653): _Remove the settings editor bug icon and always display the error inspector_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6646)](https://github.com/jupyterlab/jupyterlab/pull/6646): _Remove settings table view_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#6645)](https://github.com/jupyterlab/jupyterlab/pull/6645): _Append an empty line for test of completeness of statements_ by **[BoPeng](https://github.com/BoPeng)**
* [(#6644)](https://github.com/jupyterlab/jupyterlab/pull/6644): _Remove obsolete command use in application extension._ by **[afshin](https://github.com/afshin)**
* [(#6632)](https://github.com/jupyterlab/jupyterlab/pull/6632): _Fix missing text in #6652 which caused a sort error in js._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6630)](https://github.com/jupyterlab/jupyterlab/pull/6630): _Use stricter CSS selector for main area tab context menu so the commands don't show up elsewhere._ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6629)](https://github.com/jupyterlab/jupyterlab/pull/6629): _Fix terminal settings issues_ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6628)](https://github.com/jupyterlab/jupyterlab/pull/6628): _Remove the FAQ extension_ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6626)](https://github.com/jupyterlab/jupyterlab/pull/6626): _Fix for settings editor not hiding error messages once fixed by user_ by **[mbektasbbg](https://github.com/mbektasbbg)**
* [(#6625)](https://github.com/jupyterlab/jupyterlab/pull/6625): _Point to Jupyter docs landing page_ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6613)](https://github.com/jupyterlab/jupyterlab/pull/6613): _Notebook clones should point to the same underlying config attributes_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6607)](https://github.com/jupyterlab/jupyterlab/pull/6607): _Focus on the file browser after renaming_ by **[jtpio](https://github.com/jtpio)**
* [(#6599)](https://github.com/jupyterlab/jupyterlab/pull/6599): _Minor fixes to theme border colors._ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6597)](https://github.com/jupyterlab/jupyterlab/pull/6597): _DOC: readiness for users and devs, closes #5773_ by **[ivanov](https://github.com/ivanov)**
* [(#6587)](https://github.com/jupyterlab/jupyterlab/pull/6587): _Restyle the launcher, enable keyboard navigation_ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6586)](https://github.com/jupyterlab/jupyterlab/pull/6586): _Updates to a MainAreaWidget should propagate to the content_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6585)](https://github.com/jupyterlab/jupyterlab/pull/6585): _Lab config urls_ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6569)](https://github.com/jupyterlab/jupyterlab/pull/6569): _Separate instance tracking from widget tracking._ by **[afshin](https://github.com/afshin)**
* [(#6567)](https://github.com/jupyterlab/jupyterlab/pull/6567): _Make extension management quiet_ by **[vidartf](https://github.com/vidartf)**
* [(#6566)](https://github.com/jupyterlab/jupyterlab/pull/6566): _Move theme dataset attributes to the body element and namespace them_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6559)](https://github.com/jupyterlab/jupyterlab/pull/6559): _Render an empty widget if we don’t have a renderer for the mimebundle types._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6558)](https://github.com/jupyterlab/jupyterlab/pull/6558): _fixes #6557_ by **[telamonian](https://github.com/telamonian)**
* [(#6556)](https://github.com/jupyterlab/jupyterlab/pull/6556): _fixes #6439: underhang in highlighting of multiline selections_ by **[telamonian](https://github.com/telamonian)**
* [(#6553)](https://github.com/jupyterlab/jupyterlab/pull/6553): _Change the filebrowser home icon to a folder_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6552)](https://github.com/jupyterlab/jupyterlab/pull/6552): _Add home tooltip showing server root if available for user information._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6550)](https://github.com/jupyterlab/jupyterlab/pull/6550): _By default, add a new activity in the main area after the current widget_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6548)](https://github.com/jupyterlab/jupyterlab/pull/6548): _Handle small tabs overflowing and preserve the border bridging for the active tab_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6547)](https://github.com/jupyterlab/jupyterlab/pull/6547): _Fix jupyterlab/services README_ by **[martinRenou](https://github.com/martinRenou)**
* [(#6546)](https://github.com/jupyterlab/jupyterlab/pull/6546): _Added download attribute to Download context menu option_ by **[alexdetsch](https://github.com/alexdetsch)**
* [(#6541)](https://github.com/jupyterlab/jupyterlab/pull/6541): _Fix CSS include files_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6533)](https://github.com/jupyterlab/jupyterlab/pull/6533): _CSS Usage Cleanup_ by **[blink1073](https://github.com/blink1073)**
* [(#6526)](https://github.com/jupyterlab/jupyterlab/pull/6526): _Remove bottom border for tabBar._ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6524)](https://github.com/jupyterlab/jupyterlab/pull/6524): _Use JupyterLab's theme variables for UI component hover style._ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6522)](https://github.com/jupyterlab/jupyterlab/pull/6522): _Remove `Dialog.prompt` (use `InputDialog.getXXX` instead)_ by **[fcollonval](https://github.com/fcollonval)**
* [(#6519)](https://github.com/jupyterlab/jupyterlab/pull/6519): _Examples for vdom/vdom-extension_ by **[telamonian](https://github.com/telamonian)**
* [(#6517)](https://github.com/jupyterlab/jupyterlab/pull/6517): _Fix dialog button UX_ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6516)](https://github.com/jupyterlab/jupyterlab/pull/6516): _lazy load of vim mode_ by **[ibdafna](https://github.com/ibdafna)**
* [(#6515)](https://github.com/jupyterlab/jupyterlab/pull/6515): _Select complete multi-line statement when running run-in-console action without selection_ by **[BoPeng](https://github.com/BoPeng)**
* [(#6513)](https://github.com/jupyterlab/jupyterlab/pull/6513): _Don't append content in rendered output upon rerendering._ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6509)](https://github.com/jupyterlab/jupyterlab/pull/6509): _Update for changes to server_ by **[blink1073](https://github.com/blink1073)**
* [(#6503)](https://github.com/jupyterlab/jupyterlab/pull/6503): _Make the DefaultSession own (create and dispose) its kernel connection._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6502)](https://github.com/jupyterlab/jupyterlab/pull/6502): _Only write to filebrowser listing text if the text would change_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6500)](https://github.com/jupyterlab/jupyterlab/pull/6500): _Disable chunksSort in HtmlWebpackPlugin_ by **[mlucool](https://github.com/mlucool)**
* [(#6493)](https://github.com/jupyterlab/jupyterlab/pull/6493): _Rename "Inspector" to "Interactive Help"_ by **[ibdafna](https://github.com/ibdafna)**
* [(#6492)](https://github.com/jupyterlab/jupyterlab/pull/6492): _CSS: Remove 'overflow: visible' property from '.p-DockPanel-tabBar' class._ by **[cquah](https://github.com/cquah)**
* [(#6490)](https://github.com/jupyterlab/jupyterlab/pull/6490): _Add documentation to the Keyboard Shortcuts component in the Advanced Settings Editor to explain how to disable system default shortcuts._ by **[cquah](https://github.com/cquah)**
* [(#6489)](https://github.com/jupyterlab/jupyterlab/pull/6489): _made non-breaking space change_ by **[ibdafna](https://github.com/ibdafna)**
* [(#6486)](https://github.com/jupyterlab/jupyterlab/pull/6486): _Change keyboard shortcut for 'Close Tab' to 'Alt+W' from 'Accel+W'._ by **[cquah](https://github.com/cquah)**
* [(#6485)](https://github.com/jupyterlab/jupyterlab/pull/6485): _Rename 'Close Other Tabs' command item to 'Close All Other Tabs'._ by **[cquah](https://github.com/cquah)**
* [(#6484)](https://github.com/jupyterlab/jupyterlab/pull/6484): _Stop using py35 on Linux_ by **[blink1073](https://github.com/blink1073)**
* [(#6480)](https://github.com/jupyterlab/jupyterlab/pull/6480): _Do not wait for a console session to be ready before showing the console_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6476)](https://github.com/jupyterlab/jupyterlab/pull/6476): _added default text to inspector module_ by **[ibdafna](https://github.com/ibdafna)**
* [(#6473)](https://github.com/jupyterlab/jupyterlab/pull/6473): _Stop using base_url in workspace name._ by **[afshin](https://github.com/afshin)**
* [(#6472)](https://github.com/jupyterlab/jupyterlab/pull/6472): _Clarifies workflow to export as Reveal.js slides_ by **[duarteocarmo](https://github.com/duarteocarmo)**
* [(#6470)](https://github.com/jupyterlab/jupyterlab/pull/6470): _Remove obsolete documentation on "keymap" interface_ by **[BoPeng](https://github.com/BoPeng)**
* [(#6469)](https://github.com/jupyterlab/jupyterlab/pull/6469): _SVG namespace. Fixes #6295._ by **[zerline](https://github.com/zerline)**
* [(#6468)](https://github.com/jupyterlab/jupyterlab/pull/6468): _Stop endless Firefox redirect._ by **[afshin](https://github.com/afshin)**
* [(#6467)](https://github.com/jupyterlab/jupyterlab/pull/6467): _Terminal opens on place. Fixes #6455._ by **[zerline](https://github.com/zerline)**
* [(#6466)](https://github.com/jupyterlab/jupyterlab/pull/6466): _No more keypress/keydown confusion (no more keypress). Fixes #6452._ by **[zerline](https://github.com/zerline)**
* [(#6464)](https://github.com/jupyterlab/jupyterlab/pull/6464): _Fix minor typo in documentation_ by **[nokome](https://github.com/nokome)**
* [(#6462)](https://github.com/jupyterlab/jupyterlab/pull/6462): _More release cleanup_ by **[blink1073](https://github.com/blink1073)**
* [(#6454)](https://github.com/jupyterlab/jupyterlab/pull/6454): _Update phosphor packages to restore tab drop zone_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6451)](https://github.com/jupyterlab/jupyterlab/pull/6451): _JupyterHub integration_ by **[blink1073](https://github.com/blink1073)**
* [(#6450)](https://github.com/jupyterlab/jupyterlab/pull/6450): _More Examples Test Cleanup_ by **[blink1073](https://github.com/blink1073)**
* [(#6447)](https://github.com/jupyterlab/jupyterlab/pull/6447): _Audit keyboard shortcuts for Accel instead of Ctrl_ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6445)](https://github.com/jupyterlab/jupyterlab/pull/6445): _Release cleanup_ by **[blink1073](https://github.com/blink1073)**
* [(#6444)](https://github.com/jupyterlab/jupyterlab/pull/6444): _Modified Markdown cells' overflow to "auto"_ by **[capslockwizard](https://github.com/capslockwizard)**
* [(#6440)](https://github.com/jupyterlab/jupyterlab/pull/6440): _Upgrade to typescript 3.5.1_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6438)](https://github.com/jupyterlab/jupyterlab/pull/6438): _Update changelog to reflect 1.0a5 release, the good release._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6437)](https://github.com/jupyterlab/jupyterlab/pull/6437): _PrepublishOnly integrity check_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6433)](https://github.com/jupyterlab/jupyterlab/pull/6433): _Make kernel message typings more correct_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6430)](https://github.com/jupyterlab/jupyterlab/pull/6430): _Very rough changelog for the next alpha, alpha 4._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6429)](https://github.com/jupyterlab/jupyterlab/pull/6429): _Add version to Vega extension ID_ by **[domoritz](https://github.com/domoritz)**
* [(#6426)](https://github.com/jupyterlab/jupyterlab/pull/6426): _Audit innerHTML usage and sanitize inputs._ by **[afshin](https://github.com/afshin)**
* [(#6424)](https://github.com/jupyterlab/jupyterlab/pull/6424): _Allow Raises-exception_ by **[blink1073](https://github.com/blink1073)**
* [(#6414)](https://github.com/jupyterlab/jupyterlab/pull/6414): _Fix app example_ by **[blink1073](https://github.com/blink1073)**
* [(#6412)](https://github.com/jupyterlab/jupyterlab/pull/6412): _Rewrite Kernel Message typings and only clear cell execution prompt if needed_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6409)](https://github.com/jupyterlab/jupyterlab/pull/6409): _More granular completion_ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6407)](https://github.com/jupyterlab/jupyterlab/pull/6407): _Make selected command palette darker on hover_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#6406)](https://github.com/jupyterlab/jupyterlab/pull/6406): _Hide the Last Modified column on a narrow filebrowser._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6401)](https://github.com/jupyterlab/jupyterlab/pull/6401): _Fix file browser model autostart, simplify poll start logic._ by **[afshin](https://github.com/afshin)**
* [(#6399)](https://github.com/jupyterlab/jupyterlab/pull/6399): _Connection lost token_ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#6398)](https://github.com/jupyterlab/jupyterlab/pull/6398): _Update a number of dependencies to the current version._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6395)](https://github.com/jupyterlab/jupyterlab/pull/6395): _CSS import refactor_ by **[vidartf](https://github.com/vidartf)**
* [(#6391)](https://github.com/jupyterlab/jupyterlab/pull/6391): _Add smart copy/paste in terminal and update docs_ by **[blink1073](https://github.com/blink1073)**
* [(#6388)](https://github.com/jupyterlab/jupyterlab/pull/6388): _Refresh yarn.lock._ by **[jasongrout](https://github.com/jasongrout)**
* [(#6383)](https://github.com/jupyterlab/jupyterlab/pull/6383): _Fix scrolling down running sessions_ by **[manuzhang](https://github.com/manuzhang)**
* [(#6382)](https://github.com/jupyterlab/jupyterlab/pull/6382): _fix duplicate inspect message when typing_ by **[cnydw](https://github.com/cnydw)**
* [(#6372)](https://github.com/jupyterlab/jupyterlab/pull/6372): _Clean up handling of launch assets_ by **[blink1073](https://github.com/blink1073)**
* [(#6345)](https://github.com/jupyterlab/jupyterlab/pull/6345): _Clean coreutils package, add rate limiting utilities._ by **[afshin](https://github.com/afshin)**
* [(#6338)](https://github.com/jupyterlab/jupyterlab/pull/6338): _Add metadata for core mode packages_ by **[blink1073](https://github.com/blink1073)**
* [(#6334)](https://github.com/jupyterlab/jupyterlab/pull/6334): _Token refactor/cleanup_ by **[vidartf](https://github.com/vidartf)**
* [(#6312)](https://github.com/jupyterlab/jupyterlab/pull/6312): _Release maintenance cleanup_ by **[blink1073](https://github.com/blink1073)**
* [(#6270)](https://github.com/jupyterlab/jupyterlab/pull/6270): _[WIP] Provide a signal for opening items from a DirListing_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#6200)](https://github.com/jupyterlab/jupyterlab/pull/6200): _Add a command to enable the extension manager_ by **[jtpio](https://github.com/jtpio)**
* [(#6060)](https://github.com/jupyterlab/jupyterlab/pull/6060): _Add a clone method to ABCWidgetFactory. _ by **[Madhu94](https://github.com/Madhu94)**

##### [jupyterlab/jupyterlab-commenting](https://github.com/jupyterlab/jupyterlab-commenting)
* [(#18)](https://github.com/jupyterlab/jupyterlab-commenting/pull/18): _Press release_ by **[acu192](https://github.com/acu192)**
* [(#15)](https://github.com/jupyterlab/jupyterlab-commenting/pull/15): _Styling refactor_ by **[hoo761](https://github.com/hoo761)**

##### [jupyterlab/jupyterlab-data-explorer](https://github.com/jupyterlab/jupyterlab-data-explorer)
* [(#25)](https://github.com/jupyterlab/jupyterlab-data-explorer/pull/25): _Press release_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#24)](https://github.com/jupyterlab/jupyterlab-data-explorer/pull/24): _Add high level description to README_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#13)](https://github.com/jupyterlab/jupyterlab-data-explorer/pull/13): _Nested Data Support_ by **[saulshanabrook](https://github.com/saulshanabrook)**

##### [jupyterlab/jupyterlab-demo](https://github.com/jupyterlab/jupyterlab-demo)
* [(#86)](https://github.com/jupyterlab/jupyterlab-demo/pull/86): _Bump jupyter-leaflet to be compatible with xleaflet._ by **[ian-r-rose](https://github.com/ian-r-rose)**
* [(#83)](https://github.com/jupyterlab/jupyterlab-demo/pull/83): _Update dependencies_ by **[TheBear44](https://github.com/TheBear44)**
* [(#81)](https://github.com/jupyterlab/jupyterlab-demo/pull/81): _Drop QuantStack channel and only use conda-forge_ by **[SylvainCorlay](https://github.com/SylvainCorlay)**

##### [jupyterlab/jupyterlab-git](https://github.com/jupyterlab/jupyterlab-git)
* [(#365)](https://github.com/jupyterlab/jupyterlab-git/pull/365): _Remove psutil dependency_ by **[jaipreet-s](https://github.com/jaipreet-s)**
* [(#364)](https://github.com/jupyterlab/jupyterlab-git/pull/364): _Server API to list changed files in Git_ by **[jaipreet-s](https://github.com/jaipreet-s)**
* [(#363)](https://github.com/jupyterlab/jupyterlab-git/pull/363): _Release 0.6.0_ by **[jaipreet-s](https://github.com/jaipreet-s)**
* [(#362)](https://github.com/jupyterlab/jupyterlab-git/pull/362): _Fix push/pull to not use shell=True_ by **[jaipreet-s](https://github.com/jaipreet-s)**
* [(#361)](https://github.com/jupyterlab/jupyterlab-git/pull/361): _Bump handlebars from 4.0.12 to 4.1.2_ by **[dependabot[bot]](https://github.com/apps/dependabot)**
* [(#360)](https://github.com/jupyterlab/jupyterlab-git/pull/360): _Bump js-yaml from 3.12.0 to 3.13.1_ by **[dependabot[bot]](https://github.com/apps/dependabot)**

##### [jupyterlab/jupyterlab-github](https://github.com/jupyterlab/jupyterlab-github)
* [(#85)](https://github.com/jupyterlab/jupyterlab-github/pull/85): _Bump js-yaml from 3.12.0 to 3.13.1_ by **[dependabot[bot]](https://github.com/apps/dependabot)**

##### [jupyterlab/jupyterlab-latex](https://github.com/jupyterlab/jupyterlab-latex)
* [(#116)](https://github.com/jupyterlab/jupyterlab-latex/pull/116): _Bump notebook from 5.7.4 to 5.7.8 in /binder_ by **[dependabot[bot]](https://github.com/apps/dependabot)**

##### [jupyterlab/jupyterlab-metadata-service](https://github.com/jupyterlab/jupyterlab-metadata-service)
* [(#25)](https://github.com/jupyterlab/jupyterlab-metadata-service/pull/25): _Press release_ by **[acu192](https://github.com/acu192)**

##### [jupyterlab/jupyterlab-telemetry](https://github.com/jupyterlab/jupyterlab-telemetry)
* [(#2)](https://github.com/jupyterlab/jupyterlab-telemetry/pull/2): _Straw design document for EventLogging_ by **[yuvipanda](https://github.com/yuvipanda)**

##### [jupyterlab/jupyterlab-toc](https://github.com/jupyterlab/jupyterlab-toc)
* [(#90)](https://github.com/jupyterlab/jupyterlab-toc/pull/90): _Bump js-yaml from 3.12.0 to 3.13.1_ by **[dependabot[bot]](https://github.com/apps/dependabot)**
* [(#89)](https://github.com/jupyterlab/jupyterlab-toc/pull/89): _Update to JLab 1.0a5._ by **[marius311](https://github.com/marius311)**

##### [jupyterlab/jupyterlab_pygments](https://github.com/jupyterlab/jupyterlab_pygments)
* [(#1)](https://github.com/jupyterlab/jupyterlab_pygments/pull/1): _typo_ by **[maartenbreddels](https://github.com/maartenbreddels)**

##### [jupyterlab/jupyterlab_server](https://github.com/jupyterlab/jupyterlab_server)
* [(#70)](https://github.com/jupyterlab/jupyterlab_server/pull/70): _Cleanup for 1.0_ by **[blink1073](https://github.com/blink1073)**
* [(#69)](https://github.com/jupyterlab/jupyterlab_server/pull/69): _A 403.html file. Should close jupyterlab issue#6065_ by **[zerline](https://github.com/zerline)**
* [(#68)](https://github.com/jupyterlab/jupyterlab_server/pull/68): _Stop using base_url in workspace name._ by **[afshin](https://github.com/afshin)**
* [(#67)](https://github.com/jupyterlab/jupyterlab_server/pull/67): _v0.3.3_ by **[afshin](https://github.com/afshin)**
* [(#65)](https://github.com/jupyterlab/jupyterlab_server/pull/65): _Add frontend_url to server._ by **[afshin](https://github.com/afshin)**
* [(#64)](https://github.com/jupyterlab/jupyterlab_server/pull/64): _Friendly workspaces_ by **[afshin](https://github.com/afshin)**
* [(#63)](https://github.com/jupyterlab/jupyterlab_server/pull/63): _Parse URL-formatted metadata_id in workspace handler_ by **[thomafred](https://github.com/thomafred)**
* [(#62)](https://github.com/jupyterlab/jupyterlab_server/pull/62): _Fixed mangling of urls in theme .css files when `base_url` is set_ by **[telamonian](https://github.com/telamonian)**



## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [18]:
# Created issues
created = []
for org in tqdm(orgs):
    query = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    this_created = run_github_query(query)
    this_created['org'] = org
    created.append(this_created)
created = pd.concat(created)
created['repo'] = created['repository_url'].map(lambda a: a.split('/')[-1])

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:37<01:53, 37.68s/it]

 50%|█████     | 2/4 [01:11<01:12, 36.38s/it]

 75%|███████▌  | 3/4 [01:47<00:36, 36.29s/it]

100%|██████████| 4/4 [01:55<00:00, 27.78s/it]

In [19]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title="Issues created in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

<VegaLite 2 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [20]:
# Closed issues
closed = []
for org in tqdm(orgs):
    query = f"is:issue user:{org} closed:{start_date}..{end_date}"
    this_closed = run_github_query(query)
    this_closed['org'] = org
    closed.append(this_closed)
closed = pd.concat(closed)
closed['repo'] = closed['repository_url'].map(lambda a: a.split('/')[-1])

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:21<01:04, 21.51s/it]

 50%|█████     | 2/4 [00:59<00:52, 26.41s/it]

 75%|███████▌  | 3/4 [01:46<00:32, 32.67s/it]

100%|██████████| 4/4 [01:54<00:00, 25.17s/it]

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title="Issues closed in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

<VegaLite 2 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


### A list of recent issues

Here are the top 5 active issues in each repository in the last 30 days

In [22]:
markdown = []
for org, idata in created.groupby('org'):
    markdown.append(f"#### {org}")
    markdown.append("")
    for repourl, issues in idata.groupby('repository_url'):
        reponame, repo = repourl.split('/')[-2:]
        markdown.append(f"##### [{reponame}/{repo}](https://github.com/{reponame}/{repo})")
        for _, issue in issues.sort_values('comments', ascending=False).head(10).iterrows():
            user_name = issue['user']['login']
            user_url = issue['user']['html_url']
            issue_number = issue['number']
            issue_html = issue['html_url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[{user_name}]({user_url})**"
            markdown.append(text)
        markdown.append('')
    markdown.append('')

Markdown('\n'.join(markdown))

#### jupyter

##### [jupyter/design](https://github.com/jupyter/design)
* [(#59)](https://github.com/jupyter/design/issues/59): _Who made the "community call" logo?_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyter/docker-stacks](https://github.com/jupyter/docker-stacks)
* [(#886)](https://github.com/jupyter/docker-stacks/issues/886): _Building private versions of the stacks_ by **[richtong](https://github.com/richtong)**
* [(#883)](https://github.com/jupyter/docker-stacks/issues/883): _adding cvxopt to scipy-notebook using docker-compose_ by **[daviddoji](https://github.com/daviddoji)**
* [(#880)](https://github.com/jupyter/docker-stacks/issues/880): _Feature: restartable Jupyter in container_ by **[kafonek](https://github.com/kafonek)**
* [(#879)](https://github.com/jupyter/docker-stacks/issues/879): _add libjemalloc1 package_ by **[ivmaks](https://github.com/ivmaks)**
* [(#878)](https://github.com/jupyter/docker-stacks/issues/878): _conda environment inconsistency in scipy-notebook_ by **[philippegr](https://github.com/philippegr)**
* [(#877)](https://github.com/jupyter/docker-stacks/issues/877): _pandas.read_hdf_ by **[alanchalk](https://github.com/alanchalk)**
* [(#888)](https://github.com/jupyter/docker-stacks/issues/888): _r-notebook Dockerfile not building locally_ by **[ttimbers](https://github.com/ttimbers)**
* [(#887)](https://github.com/jupyter/docker-stacks/issues/887): _add scikit-monaco package_ by **[daviddoji](https://github.com/daviddoji)**
* [(#885)](https://github.com/jupyter/docker-stacks/issues/885): _Permission denied when mounting local folders_ by **[ejgutierrez74](https://github.com/ejgutierrez74)**
* [(#884)](https://github.com/jupyter/docker-stacks/issues/884): _Exception: Java gateway process exited before sending its port number_ by **[yonghaihung](https://github.com/yonghaihung)**

##### [jupyter/enterprise_gateway](https://github.com/jupyter/enterprise_gateway)
* [(#683)](https://github.com/jupyter/enterprise_gateway/issues/683): _Request to Kernel spawn fails even if the KG_REQUEST_TIMEOUT and KG_CONNECT_TIMEOUT are increased_ by **[suryag10](https://github.com/suryag10)**
* [(#678)](https://github.com/jupyter/enterprise_gateway/issues/678): _Launching ipython kernel with custom arguments_ by **[IMAM9AIS](https://github.com/IMAM9AIS)**
* [(#676)](https://github.com/jupyter/enterprise_gateway/issues/676): _Exchange user specific files between EG kernel and Notebook container, and restrict access of files between Notebook containers._ by **[ArindamHalder7](https://github.com/ArindamHalder7)**

##### [jupyter/jupyter-book](https://github.com/jupyter/jupyter-book)
* [(#213)](https://github.com/jupyter/jupyter-book/issues/213): _live reload not working with `make serve`_ by **[thammegowda](https://github.com/thammegowda)**
* [(#205)](https://github.com/jupyter/jupyter-book/issues/205): _docker file for build-container?_ by **[kkmann](https://github.com/kkmann)**
* [(#221)](https://github.com/jupyter/jupyter-book/issues/221): _scroll bar with many "hidecode" tags_ by **[joergbrech](https://github.com/joergbrech)**
* [(#220)](https://github.com/jupyter/jupyter-book/issues/220): _Cannot publish book locally using docker_ by **[kurniawano](https://github.com/kurniawano)**
* [(#204)](https://github.com/jupyter/jupyter-book/issues/204): _Metadata includes legacy literal - publisher as "Data 100"_ by **[psychemedia](https://github.com/psychemedia)**
* [(#219)](https://github.com/jupyter/jupyter-book/issues/219): _Why do we have "content" directory inside the "_site"?_ by **[thammegowda](https://github.com/thammegowda)**
* [(#212)](https://github.com/jupyter/jupyter-book/issues/212): _Enable hiding (with visbility toggle) of markdown cells similar to code cells_ by **[BENR0](https://github.com/BENR0)**
* [(#211)](https://github.com/jupyter/jupyter-book/issues/211): _Double check installation dependencies_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyter/jupyter_client](https://github.com/jupyter/jupyter_client)
* [(#445)](https://github.com/jupyter/jupyter_client/issues/445): _Reference kernelspec resources from kernel?_ by **[devinrsmith](https://github.com/devinrsmith)**
* [(#442)](https://github.com/jupyter/jupyter_client/issues/442): _is_complete_reply status conflicts with general reply status_ by **[jasongrout](https://github.com/jasongrout)**
* [(#446)](https://github.com/jupyter/jupyter_client/issues/446): _Supporting the Debug Adapter Protocol_ by **[JohanMabille](https://github.com/JohanMabille)**
* [(#443)](https://github.com/jupyter/jupyter_client/issues/443): _execute_reply has extra fields even when status is error or abort_ by **[jasongrout](https://github.com/jasongrout)**

##### [jupyter/jupyter_core](https://github.com/jupyter/jupyter_core)
* [(#156)](https://github.com/jupyter/jupyter_core/issues/156): _4.5.0 fails to install_ by **[esselius](https://github.com/esselius)**

##### [jupyter/jupyter_kernel_test](https://github.com/jupyter/jupyter_kernel_test)
* [(#51)](https://github.com/jupyter/jupyter_kernel_test/issues/51): _tests broken as of master_ by **[anntzer](https://github.com/anntzer)**

##### [jupyter/jupyter_server](https://github.com/jupyter/jupyter_server)
* [(#53)](https://github.com/jupyter/jupyter_server/issues/53): _Sync Notebook and Jupyter Server._ by **[Zsailer](https://github.com/Zsailer)**
* [(#55)](https://github.com/jupyter/jupyter_server/issues/55): _Configure extensions from server CLI._ by **[Zsailer](https://github.com/Zsailer)**
* [(#51)](https://github.com/jupyter/jupyter_server/issues/51): _Support --no-browser_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#50)](https://github.com/jupyter/jupyter_server/issues/50): _[DISCUSS] Pluggable user token creation/validation_ by **[echarles](https://github.com/echarles)**
* [(#58)](https://github.com/jupyter/jupyter_server/issues/58): _Issue when using directory named "checkpoints" or "trust"_ by **[xuchenCN](https://github.com/xuchenCN)**
* [(#57)](https://github.com/jupyter/jupyter_server/issues/57): _The readme's link is broken?_ by **[Dustyposa](https://github.com/Dustyposa)**

##### [jupyter/nbconvert](https://github.com/jupyter/nbconvert)
* [(#1031)](https://github.com/jupyter/nbconvert/issues/1031): _Gnarly proof-of-concept PDF generation with QtWebEngine_ by **[bollwyvl](https://github.com/bollwyvl)**
* [(#1045)](https://github.com/jupyter/nbconvert/issues/1045): _NBConvert 6.0 Planning_ by **[MSeal](https://github.com/MSeal)**
* [(#1032)](https://github.com/jupyter/nbconvert/issues/1032): _Optional store_history for ExecutePreprocessor.preprocess_cell_ by **[wwitzel](https://github.com/wwitzel)**
* [(#1052)](https://github.com/jupyter/nbconvert/issues/1052): _NBConvert 5.6 Release_ by **[MSeal](https://github.com/MSeal)**
* [(#1037)](https://github.com/jupyter/nbconvert/issues/1037): _Issue rendering markdown cells to HTML when they contain <div> tags_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1043)](https://github.com/jupyter/nbconvert/issues/1043): _Document "remove_input_tag" traitlet_ by **[choldgraf](https://github.com/choldgraf)**
* [(#1041)](https://github.com/jupyter/nbconvert/issues/1041): _How to create thumbnails from ipynb files or jupyter notebooks_ by **[vasu0494](https://github.com/vasu0494)**
* [(#1033)](https://github.com/jupyter/nbconvert/issues/1033): _Deprecated exporter_locator always imported_ by **[SylvainCorlay](https://github.com/SylvainCorlay)**

##### [jupyter/nbdime](https://github.com/jupyter/nbdime)
* [(#480)](https://github.com/jupyter/nbdime/issues/480): _NBDime REST API changes for git diffs_ by **[jaipreet-s](https://github.com/jaipreet-s)**
* [(#478)](https://github.com/jupyter/nbdime/issues/478): _use nbdime as a clever git filter_ by **[qmarcou](https://github.com/qmarcou)**
* [(#479)](https://github.com/jupyter/nbdime/issues/479): _Enabling extension with `sys-prefix` fails_ by **[agoose77](https://github.com/agoose77)**

##### [jupyter/nbgrader](https://github.com/jupyter/nbgrader)
* [(#1097)](https://github.com/jupyter/nbgrader/issues/1097): _Impersonation of other users via the exchange_ by **[rkdarst](https://github.com/rkdarst)**
* [(#1148)](https://github.com/jupyter/nbgrader/issues/1148): _Running "create assignment" as a standalone module_ by **[psychemedia](https://github.com/psychemedia)**
* [(#1158)](https://github.com/jupyter/nbgrader/issues/1158): _KeyError when calling `nbgrader feedback`_ by **[b-stone](https://github.com/b-stone)**
* [(#1104)](https://github.com/jupyter/nbgrader/issues/1104): _Timestamps can be altered using the filesystem exchange_ by **[rkdarst](https://github.com/rkdarst)**
* [(#1141)](https://github.com/jupyter/nbgrader/issues/1141): _Possible to fetch feedback for only some notebooks in an assignment_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1154)](https://github.com/jupyter/nbgrader/issues/1154): _Assignment generation: adding: raise NotImplementedError()_ by **[psychemedia](https://github.com/psychemedia)**
* [(#1153)](https://github.com/jupyter/nbgrader/issues/1153): _Randomising Values in questions and tests as part of assignment process_ by **[psychemedia](https://github.com/psychemedia)**
* [(#1130)](https://github.com/jupyter/nbgrader/issues/1130): _Implement a generic API to support custom exchanges_ by **[jhamrick](https://github.com/jhamrick)**
* [(#1157)](https://github.com/jupyter/nbgrader/issues/1157): _Exclude students when autograding_ by **[rkdarst](https://github.com/rkdarst)**
* [(#1095)](https://github.com/jupyter/nbgrader/issues/1095): _Use db for exchange mechanism_ by **[ryanlovett](https://github.com/ryanlovett)**

##### [jupyter/nbviewer](https://github.com/jupyter/nbviewer)
* [(#835)](https://github.com/jupyter/nbviewer/issues/835): _Fully formed Github repo urls leading to 400 error_ by **[fperez](https://github.com/fperez)**
* [(#837)](https://github.com/jupyter/nbviewer/issues/837): _Docker image : enable extensions_ by **[dbeniamine](https://github.com/dbeniamine)**
* [(#836)](https://github.com/jupyter/nbviewer/issues/836): _nbviewer error displaying output from IPython.display.Javascript when using lib=_ by **[tony-johnson](https://github.com/tony-johnson)**
* [(#834)](https://github.com/jupyter/nbviewer/issues/834): _Update the FAQ to point people to the Discourse_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyter/notebook](https://github.com/jupyter/notebook)
* [(#4698)](https://github.com/jupyter/notebook/issues/4698): _Release v6.0_ by **[choldgraf](https://github.com/choldgraf)**
* [(#4680)](https://github.com/jupyter/notebook/issues/4680): _Latest preact-compat release breaks notebook._ by **[ssanderson](https://github.com/ssanderson)**
* [(#4640)](https://github.com/jupyter/notebook/issues/4640): _Contents API sort of works for windows-style C:\ paths_ by **[jasongrout](https://github.com/jasongrout)**
* [(#4663)](https://github.com/jupyter/notebook/issues/4663): _is there sentence case in Jupyter Notebook?_ by **[Iorent](https://github.com/Iorent)**
* [(#4665)](https://github.com/jupyter/notebook/issues/4665): _how to make edit section include front extension ?_ by **[autodataming](https://github.com/autodataming)**
* [(#4644)](https://github.com/jupyter/notebook/issues/4644): _SyntaxError with Python 2_ by **[Elizaveta239](https://github.com/Elizaveta239)**
* [(#4645)](https://github.com/jupyter/notebook/issues/4645): _keep checkpoints file aside (c.FileCheckpoints)_ by **[gsemet](https://github.com/gsemet)**
* [(#4656)](https://github.com/jupyter/notebook/issues/4656): _Error: Unable to create process_ by **[debsdo](https://github.com/debsdo)**
* [(#4639)](https://github.com/jupyter/notebook/issues/4639): _Ability to provide stdin to nbconvert_ by **[youngmit](https://github.com/youngmit)**
* [(#4673)](https://github.com/jupyter/notebook/issues/4673): _Pin Jupyter Notebook icon onto system tray_ by **[Iorent](https://github.com/Iorent)**

##### [jupyter/qtconsole](https://github.com/jupyter/qtconsole)
* [(#345)](https://github.com/jupyter/qtconsole/issues/345): _No module named 'aip'_ by **[jenny269](https://github.com/jenny269)**

##### [jupyter/repo2docker](https://github.com/jupyter/repo2docker)
* [(#686)](https://github.com/jupyter/repo2docker/issues/686): _Add support for packagcompiler for julia support_ by **[jlperla](https://github.com/jlperla)**
* [(#701)](https://github.com/jupyter/repo2docker/issues/701): _Make the names of r2d generated images more readable_ by **[sgibson91](https://github.com/sgibson91)**
* [(#707)](https://github.com/jupyter/repo2docker/issues/707): _The need for speed_ by **[betatim](https://github.com/betatim)**
* [(#702)](https://github.com/jupyter/repo2docker/issues/702): _Implement a GUI that builds repo2docker config files_ by **[choldgraf](https://github.com/choldgraf)**
* [(#700)](https://github.com/jupyter/repo2docker/issues/700): _man pages on the bionic master images are deactivated_ by **[ocordes](https://github.com/ocordes)**
* [(#691)](https://github.com/jupyter/repo2docker/issues/691): _case sensitivity issue with postBuild_ by **[minrk](https://github.com/minrk)**
* [(#687)](https://github.com/jupyter/repo2docker/issues/687): _Allow BuildPack to use a custom template in .render()_ by **[Xarthisius](https://github.com/Xarthisius)**
* [(#698)](https://github.com/jupyter/repo2docker/issues/698): _In a Python 2 repository `python` gives you Python 3_ by **[betatim](https://github.com/betatim)**
* [(#696)](https://github.com/jupyter/repo2docker/issues/696): _Setting r2d.user_id = 0 doesn't cause exception_ by **[yuvipanda](https://github.com/yuvipanda)**


#### jupyter-widgets

##### [jupyter-widgets/ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet)
* [(#373)](https://github.com/jupyter-widgets/ipyleaflet/issues/373): _In the geojson layer the on_hover and on_click events calls have very different arguments._ by **[dwr-psandhu](https://github.com/dwr-psandhu)**
* [(#367)](https://github.com/jupyter-widgets/ipyleaflet/issues/367): _Changing position of some controls doesn't work_ by **[kgiacobbi](https://github.com/kgiacobbi)**
* [(#379)](https://github.com/jupyter-widgets/ipyleaflet/issues/379): _Add the redraw method for TileLayer_ by **[kgiacobbi](https://github.com/kgiacobbi)**
* [(#377)](https://github.com/jupyter-widgets/ipyleaflet/issues/377): _Enable CI_ by **[martinRenou](https://github.com/martinRenou)**
* [(#372)](https://github.com/jupyter-widgets/ipyleaflet/issues/372): _Select tile layers programmatically_ by **[deeplook](https://github.com/deeplook)**
* [(#370)](https://github.com/jupyter-widgets/ipyleaflet/issues/370): _Displayed event issues_ by **[martinRenou](https://github.com/martinRenou)**
* [(#369)](https://github.com/jupyter-widgets/ipyleaflet/issues/369): _Can't interact with ipywidgets.Tab in a WidgetControl_ by **[kgiacobbi](https://github.com/kgiacobbi)**

##### [jupyter-widgets/ipywidgets](https://github.com/jupyter-widgets/ipywidgets)
* [(#2417)](https://github.com/jupyter-widgets/ipywidgets/issues/2417): _Mention ipython_blocking in the async docs_ by **[jasongrout](https://github.com/jasongrout)**
* [(#2443)](https://github.com/jupyter-widgets/ipywidgets/issues/2443): _Bounded Float / Int text defaults_ by **[lheagy](https://github.com/lheagy)**
* [(#2439)](https://github.com/jupyter-widgets/ipywidgets/issues/2439): _Error While Embedding Jupyter Widgets in Other Contexts than the Notebook_ by **[Pabla007](https://github.com/Pabla007)**
* [(#2437)](https://github.com/jupyter-widgets/ipywidgets/issues/2437): _Cannot change the width of the widget "Password"_ by **[piyueh](https://github.com/piyueh)**
* [(#2427)](https://github.com/jupyter-widgets/ipywidgets/issues/2427): _interact() does not update the values inside the given function (Live Audio)_ by **[JohnIssue](https://github.com/JohnIssue)**
* [(#2426)](https://github.com/jupyter-widgets/ipywidgets/issues/2426): _Gettiing issues while using SQL kernel and trying to display a table_ by **[KaynRyu](https://github.com/KaynRyu)**
* [(#2419)](https://github.com/jupyter-widgets/ipywidgets/issues/2419): _How to remove Matplotlib plots flickering when refreshing the output often_ by **[bybyte](https://github.com/bybyte)**
* [(#2446)](https://github.com/jupyter-widgets/ipywidgets/issues/2446): _Deprecation of widgets.Text.on_submit_ by **[janfreyberg](https://github.com/janfreyberg)**
* [(#2444)](https://github.com/jupyter-widgets/ipywidgets/issues/2444): _FloatText widget replaces values as Int _ by **[denphi](https://github.com/denphi)**
* [(#2440)](https://github.com/jupyter-widgets/ipywidgets/issues/2440): _FloatSlider static html export not working as expected_ by **[Boomer91](https://github.com/Boomer91)**

##### [jupyter-widgets/jupyter-sphinx](https://github.com/jupyter-widgets/jupyter-sphinx)
* [(#50)](https://github.com/jupyter-widgets/jupyter-sphinx/issues/50): _Move to Jupyter GitHub organization?_ by **[SylvainCorlay](https://github.com/SylvainCorlay)**
* [(#48)](https://github.com/jupyter-widgets/jupyter-sphinx/issues/48): _handle stderr_ by **[akhmerov](https://github.com/akhmerov)**
* [(#54)](https://github.com/jupyter-widgets/jupyter-sphinx/issues/54): _maint: use pathlib_ by **[akhmerov](https://github.com/akhmerov)**

##### [jupyter-widgets/jupyterlab-sidecar](https://github.com/jupyter-widgets/jupyterlab-sidecar)
* [(#18)](https://github.com/jupyter-widgets/jupyterlab-sidecar/issues/18): _jupyterlab-sidecar compatibility with jupyter 1.0_ by **[downiec](https://github.com/downiec)**

##### [jupyter-widgets/pythreejs](https://github.com/jupyter-widgets/pythreejs)
* [(#274)](https://github.com/jupyter-widgets/pythreejs/issues/274): _Mouse control gets stuck and objects disappear when using multiple Renderers inside ipywidgets.interact_ by **[t-suzuki](https://github.com/t-suzuki)**
* [(#273)](https://github.com/jupyter-widgets/pythreejs/issues/273): _How do you use loaders?_ by **[aliddell](https://github.com/aliddell)**
* [(#279)](https://github.com/jupyter-widgets/pythreejs/issues/279): _PythreeJs cannot render more than 1000 objects if you use example helper_ by **[TakodaS](https://github.com/TakodaS)**
* [(#275)](https://github.com/jupyter-widgets/pythreejs/issues/275): _new release of jupyter-sphinx_ by **[akhmerov](https://github.com/akhmerov)**

##### [jupyter-widgets/tutorial](https://github.com/jupyter-widgets/tutorial)
* [(#27)](https://github.com/jupyter-widgets/tutorial/issues/27): _LinkError on ipywebrtc while building environment with conda_ by **[akielbowicz](https://github.com/akielbowicz)**
* [(#40)](https://github.com/jupyter-widgets/tutorial/issues/40): _bqplot version_ by **[martinRenou](https://github.com/martinRenou)**


#### jupyterhub

##### [jupyterhub/alabaster-jupyterhub](https://github.com/jupyterhub/alabaster-jupyterhub)
* [(#15)](https://github.com/jupyterhub/alabaster-jupyterhub/issues/15): _Make the right TOC scrollable_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyterhub/binderhub](https://github.com/jupyterhub/binderhub)
* [(#871)](https://github.com/jupyterhub/binderhub/issues/871): _Terminal doesn't activate the Binder environment in JupyterLab_ by **[choldgraf](https://github.com/choldgraf)**
* [(#854)](https://github.com/jupyterhub/binderhub/issues/854): _Build pods and the use of the z2jh user-scheduler_ by **[consideRatio](https://github.com/consideRatio)**
* [(#850)](https://github.com/jupyterhub/binderhub/issues/850): _Add anchors to filepath?_ by **[kteich-oreilly](https://github.com/kteich-oreilly)**
* [(#880)](https://github.com/jupyterhub/binderhub/issues/880): _Notes on updating the user interface_ by **[betatim](https://github.com/betatim)**
* [(#864)](https://github.com/jupyterhub/binderhub/issues/864): _[feature request] Automatic docker image pull on each node_ by **[ltetrel](https://github.com/ltetrel)**
* [(#862)](https://github.com/jupyterhub/binderhub/issues/862): _Support a header.html and a footer.html_ by **[choldgraf](https://github.com/choldgraf)**
* [(#855)](https://github.com/jupyterhub/binderhub/issues/855): _Optimize the user-schedulers conf for image locality_ by **[consideRatio](https://github.com/consideRatio)**
* [(#852)](https://github.com/jupyterhub/binderhub/issues/852): _Give users feedback on image transfer progress_ by **[betatim](https://github.com/betatim)**
* [(#875)](https://github.com/jupyterhub/binderhub/issues/875): _Add sharing widget to loading page_ by **[betatim](https://github.com/betatim)**
* [(#874)](https://github.com/jupyterhub/binderhub/issues/874): _Advertise hub's current usage and capacity_ by **[betatim](https://github.com/betatim)**

##### [jupyterhub/chartpress](https://github.com/jupyterhub/chartpress)
* [(#38)](https://github.com/jupyterhub/chartpress/issues/38): _using chartpress with pre-commit_ by **[rokroskar](https://github.com/rokroskar)**
* [(#37)](https://github.com/jupyterhub/chartpress/issues/37): _Using ChartPress for releases with GitHub Actions_ by **[manics](https://github.com/manics)**

##### [jupyterhub/configurable-http-proxy](https://github.com/jupyterhub/configurable-http-proxy)
* [(#207)](https://github.com/jupyterhub/configurable-http-proxy/issues/207): _Large header support for node.js_ by **[brianv0](https://github.com/brianv0)**
* [(#205)](https://github.com/jupyterhub/configurable-http-proxy/issues/205): _Command line option for custom headers_ by **[ivan-gomes](https://github.com/ivan-gomes)**

##### [jupyterhub/dockerspawner](https://github.com/jupyterhub/dockerspawner)
* [(#313)](https://github.com/jupyterhub/dockerspawner/issues/313): _pull_image fails if docker regsitry not on port 443_ by **[timhughes](https://github.com/timhughes)**
* [(#314)](https://github.com/jupyterhub/dockerspawner/issues/314): _Tests fail against  jupyterhub 1.0.0_ by **[timhughes](https://github.com/timhughes)**
* [(#312)](https://github.com/jupyterhub/dockerspawner/issues/312): _Can't change UID and GID in spawning containers_ by **[m-alekseev](https://github.com/m-alekseev)**
* [(#311)](https://github.com/jupyterhub/dockerspawner/issues/311): _'The absolute minimal JupyterHub user image' does not work with dockerspawner.SystemUserSpawner (?)_ by **[kozo2](https://github.com/kozo2)**

##### [jupyterhub/dummyauthenticator](https://github.com/jupyterhub/dummyauthenticator)
* [(#8)](https://github.com/jupyterhub/dummyauthenticator/issues/8): _It not works after setting as README._ by **[Google-Oriented-Coder](https://github.com/Google-Oriented-Coder)**

##### [jupyterhub/firstuseauthenticator](https://github.com/jupyterhub/firstuseauthenticator)
* [(#19)](https://github.com/jupyterhub/firstuseauthenticator/issues/19): _Set minimum length on passwords_ by **[yuvipanda](https://github.com/yuvipanda)**

##### [jupyterhub/jupyter-rsession-proxy](https://github.com/jupyterhub/jupyter-rsession-proxy)
* [(#70)](https://github.com/jupyterhub/jupyter-rsession-proxy/issues/70): _Could we make 'Shiny' menu item as optional?_ by **[standby7878](https://github.com/standby7878)**

##### [jupyterhub/jupyter-server-proxy](https://github.com/jupyterhub/jupyter-server-proxy)
* [(#133)](https://github.com/jupyterhub/jupyter-server-proxy/issues/133): _tomcat process_ by **[djangoliv](https://github.com/djangoliv)**

##### [jupyterhub/jupyterhub](https://github.com/jupyterhub/jupyterhub)
* [(#2598)](https://github.com/jupyterhub/jupyterhub/issues/2598): _add configurable limit to the kernel runtime _ by **[rochaporto](https://github.com/rochaporto)**
* [(#2576)](https://github.com/jupyterhub/jupyterhub/issues/2576): _permission denied when acl is configured_ by **[eylenth](https://github.com/eylenth)**
* [(#2597)](https://github.com/jupyterhub/jupyterhub/issues/2597): _how do you connect a javascript app to a jupyterhub kernel?_ by **[bionicles](https://github.com/bionicles)**
* [(#2607)](https://github.com/jupyterhub/jupyterhub/issues/2607): _Unable to access API with generated API key_ by **[caseyzak24](https://github.com/caseyzak24)**
* [(#2602)](https://github.com/jupyterhub/jupyterhub/issues/2602): _Redirect loop trying to oauth with ORCiD _ by **[brayanrodbajo](https://github.com/brayanrodbajo)**
* [(#2608)](https://github.com/jupyterhub/jupyterhub/issues/2608): _500: Internal server error_ by **[payal4143](https://github.com/payal4143)**
* [(#2604)](https://github.com/jupyterhub/jupyterhub/issues/2604): _access other servers_ by **[mathematicalmichael](https://github.com/mathematicalmichael)**
* [(#2580)](https://github.com/jupyterhub/jupyterhub/issues/2580): _Using SwarmSpawner can't pass arguments to change NB_USER NB_UID etc_ by **[m-alekseev](https://github.com/m-alekseev)**
* [(#2592)](https://github.com/jupyterhub/jupyterhub/issues/2592): _JupyterHub gets 'stuck' trying to start for >2min_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#2589)](https://github.com/jupyterhub/jupyterhub/issues/2589): _Named server string sanitization_ by **[vilhelmen](https://github.com/vilhelmen)**

##### [jupyterhub/jupyterhub-deploy-docker](https://github.com/jupyterhub/jupyterhub-deploy-docker)
* [(#86)](https://github.com/jupyterhub/jupyterhub-deploy-docker/issues/86): _Docker volumes permission denied_ by **[SanderGn](https://github.com/SanderGn)**

##### [jupyterhub/jupyterlab-hub](https://github.com/jupyterhub/jupyterlab-hub)
* [(#92)](https://github.com/jupyterhub/jupyterlab-hub/issues/92): _Repo still relevant?_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#91)](https://github.com/jupyterhub/jupyterlab-hub/issues/91): _jupyterlab-hub include page_config.json check and provide help finding JupyterHub_ by **[NumesSanguis](https://github.com/NumesSanguis)**
* [(#90)](https://github.com/jupyterhub/jupyterlab-hub/issues/90): _Too strict version check - hub-extension@1.0.0-alpha.6 not installing but 0.12.0 does_ by **[NumesSanguis](https://github.com/NumesSanguis)**
* [(#89)](https://github.com/jupyterhub/jupyterlab-hub/issues/89): _Preventing implicit spawn for user because last spawn failed_ by **[maxpv](https://github.com/maxpv)**

##### [jupyterhub/kubespawner](https://github.com/jupyterhub/kubespawner)
* [(#324)](https://github.com/jupyterhub/kubespawner/issues/324): _username for the form "aaa-bbb" gets PVC like: "claim-aaa-e2bbb-XXXX"_ by **[gsemet](https://github.com/gsemet)**
* [(#328)](https://github.com/jupyterhub/kubespawner/issues/328): _Spawn Failed Message: Harmless Error but Could Turn Off Novice Users_ by **[julienchastang](https://github.com/julienchastang)**
* [(#325)](https://github.com/jupyterhub/kubespawner/issues/325): _z2jh's single.extra-env does not seem to be consummed_ by **[gsemet](https://github.com/gsemet)**
* [(#326)](https://github.com/jupyterhub/kubespawner/issues/326): _Issue Creating Ephemeral User Pods_ by **[afeiszli](https://github.com/afeiszli)**

##### [jupyterhub/ldapauthenticator](https://github.com/jupyterhub/ldapauthenticator)
* [(#125)](https://github.com/jupyterhub/ldapauthenticator/issues/125): _How to pass domain/username through LDAP_ by **[jljs](https://github.com/jljs)**
* [(#124)](https://github.com/jupyterhub/ldapauthenticator/issues/124): _LDAP timeout?_ by **[adriendelsalle](https://github.com/adriendelsalle)**

##### [jupyterhub/ltiauthenticator](https://github.com/jupyterhub/ltiauthenticator)
* [(#19)](https://github.com/jupyterhub/ltiauthenticator/issues/19): _Incompatibilities and hanging tljh-config reload_ by **[wmunters](https://github.com/wmunters)**

##### [jupyterhub/mybinder.org-deploy](https://github.com/jupyterhub/mybinder.org-deploy)
* [(#1021)](https://github.com/jupyterhub/mybinder.org-deploy/issues/1021): _OVH cluster scale up plan_ by **[betatim](https://github.com/betatim)**
* [(#1038)](https://github.com/jupyterhub/mybinder.org-deploy/issues/1038): _Pre-pull popular images on nodes to make mybinder.org faster_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#999)](https://github.com/jupyterhub/mybinder.org-deploy/issues/999): _Federation redirector setup_ by **[betatim](https://github.com/betatim)**
* [(#1036)](https://github.com/jupyterhub/mybinder.org-deploy/issues/1036): _Document where the GitHub tokens come from_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#985)](https://github.com/jupyterhub/mybinder.org-deploy/issues/985): _Use Google Conntainer Registry's docker hub mirror_ by **[betatim](https://github.com/betatim)**
* [(#1023)](https://github.com/jupyterhub/mybinder.org-deploy/issues/1023): _Deploy to all production clusters, then test them_ by **[betatim](https://github.com/betatim)**
* [(#998)](https://github.com/jupyterhub/mybinder.org-deploy/issues/998): _Updated project owners for binder-prod on google cloud_ by **[betatim](https://github.com/betatim)**

##### [jupyterhub/nbgitpuller](https://github.com/jupyterhub/nbgitpuller)
* [(#83)](https://github.com/jupyterhub/nbgitpuller/issues/83): _Add url parameter to prevent redirection after syncing_ by **[parmentelat](https://github.com/parmentelat)**
* [(#91)](https://github.com/jupyterhub/nbgitpuller/issues/91): _turning an existing dir into a git repo_ by **[parmentelat](https://github.com/parmentelat)**
* [(#85)](https://github.com/jupyterhub/nbgitpuller/issues/85): _Block insecure ways of pulling from private repositories by default_ by **[yuvipanda](https://github.com/yuvipanda)**
* [(#87)](https://github.com/jupyterhub/nbgitpuller/issues/87): _Use `dev0` for development version of nbgitpuller_ by **[choldgraf](https://github.com/choldgraf)**
* [(#84)](https://github.com/jupyterhub/nbgitpuller/issues/84): _Automatically set user.name & user.email if necessary_ by **[parmentelat](https://github.com/parmentelat)**
* [(#96)](https://github.com/jupyterhub/nbgitpuller/issues/96): _Add mybinder to the link generator_ by **[fmaussion](https://github.com/fmaussion)**
* [(#95)](https://github.com/jupyterhub/nbgitpuller/issues/95): _Have a --force option on gitpuller_ by **[clancychilds](https://github.com/clancychilds)**
* [(#94)](https://github.com/jupyterhub/nbgitpuller/issues/94): _need a way to define final redirect url_ by **[parmentelat](https://github.com/parmentelat)**
* [(#90)](https://github.com/jupyterhub/nbgitpuller/issues/90): _`repo_dir = repo.split('/')[-1]`_ by **[parmentelat](https://github.com/parmentelat)**

##### [jupyterhub/oauthenticator](https://github.com/jupyterhub/oauthenticator)
* [(#271)](https://github.com/jupyterhub/oauthenticator/issues/271): _302 Redirect issue _ by **[hedgehog-millenium](https://github.com/hedgehog-millenium)**
* [(#273)](https://github.com/jupyterhub/oauthenticator/issues/273): _Generic oauth handler does not include client_id and client_secret in POST request_ by **[chucklay](https://github.com/chucklay)**
* [(#265)](https://github.com/jupyterhub/oauthenticator/issues/265): _GitHub authentication for Organization Teams_ by **[scottyhq](https://github.com/scottyhq)**
* [(#266)](https://github.com/jupyterhub/oauthenticator/issues/266): _Using Auth0 usernames instead of email addresses_ by **[JuanCab](https://github.com/JuanCab)**

##### [jupyterhub/systemdspawner](https://github.com/jupyterhub/systemdspawner)
* [(#52)](https://github.com/jupyterhub/systemdspawner/issues/52): _Add USERHOME placeholder_ by **[mxjeff](https://github.com/mxjeff)**

##### [jupyterhub/team-compass](https://github.com/jupyterhub/team-compass)
* [(#164)](https://github.com/jupyterhub/team-compass/issues/164): _mybinder.org user survey_ by **[betatim](https://github.com/betatim)**
* [(#163)](https://github.com/jupyterhub/team-compass/issues/163): _Transfer over domain rights of `mybinder.org` to the Jupyter account_ by **[choldgraf](https://github.com/choldgraf)**
* [(#172)](https://github.com/jupyterhub/team-compass/issues/172): _Blog post: Zenodo integration_ by **[choldgraf](https://github.com/choldgraf)**
* [(#171)](https://github.com/jupyterhub/team-compass/issues/171): _Binder federation blog post_ by **[betatim](https://github.com/betatim)**
* [(#170)](https://github.com/jupyterhub/team-compass/issues/170): _Add a place for team discussion to the Community Forum_ by **[choldgraf](https://github.com/choldgraf)**
* [(#169)](https://github.com/jupyterhub/team-compass/issues/169): _Review Request: BinderHub chapter in the Turing Way_ by **[sgibson91](https://github.com/sgibson91)**
* [(#165)](https://github.com/jupyterhub/team-compass/issues/165): _Job description and duties for a "Binder fellow"?_ by **[choldgraf](https://github.com/choldgraf)**

##### [jupyterhub/the-littlest-jupyterhub](https://github.com/jupyterhub/the-littlest-jupyterhub)
* [(#351)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/351): _server is slow to start then gives timeout error_ by **[lachlancampbell](https://github.com/lachlancampbell)**
* [(#375)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/375): _Change password doesn't work_ by **[efedorov-dart](https://github.com/efedorov-dart)**
* [(#349)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/349): _add jupyter-* to some group_ by **[gsemet](https://github.com/gsemet)**
* [(#346)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/346): _Intermittent failure to install when using Ansible playbook_ by **[lachlancampbell](https://github.com/lachlancampbell)**
* [(#345)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/345): _Automatic HTTPS setting doesn't work_ by **[lucas-mior](https://github.com/lucas-mior)**
* [(#369)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/369): _PyQt5 issue_ by **[GerardoLopez](https://github.com/GerardoLopez)**
* [(#378)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/378): _admin-user-name can not be capitalized_ by **[burkett123](https://github.com/burkett123)**
* [(#377)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/377): _cut and paste problem in TLJH release 0.1 guide_ by **[burkett123](https://github.com/burkett123)**
* [(#361)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/361): _Jupyter Hub doesn't start if auth state is enabled_ by **[jmartinc89](https://github.com/jmartinc89)**
* [(#353)](https://github.com/jupyterhub/the-littlest-jupyterhub/issues/353): _Deleting accounts from the command line_ by **[JuanCab](https://github.com/JuanCab)**

##### [jupyterhub/traefik-proxy](https://github.com/jupyterhub/traefik-proxy)
* [(#58)](https://github.com/jupyterhub/traefik-proxy/issues/58): _[ASK] Traefik configuration example for multiple node server_ by **[sundh4](https://github.com/sundh4)**

##### [jupyterhub/zero-to-jupyterhub-k8s](https://github.com/jupyterhub/zero-to-jupyterhub-k8s)
* [(#1310)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1310): _External services not being passed to hub configuration_ by **[rschroll](https://github.com/rschroll)**
* [(#1314)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1314): _Not setting explicit host on Ingress_ by **[nfds89](https://github.com/nfds89)**
* [(#1311)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1311): _How to set env variable after the server has started?_ by **[frouzbeh](https://github.com/frouzbeh)**
* [(#1302)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1302): _Ability to pass extra command line options to configurable-http-proxy_ by **[ivan-gomes](https://github.com/ivan-gomes)**
* [(#1305)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1305): _How does image-awaiter and image-puller interact?_ by **[Lastcysa](https://github.com/Lastcysa)**
* [(#1300)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1300): _User-scheduler doesn't work with rancher/local-path-provisioner_ by **[manics](https://github.com/manics)**
* [(#1297)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1297): _Native authenticator by default_ by **[consideRatio](https://github.com/consideRatio)**
* [(#1315)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1315): _hub pod stuck in pending_ by **[Dreeseaw](https://github.com/Dreeseaw)**
* [(#1313)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1313): _Release 0.9_ by **[consideRatio](https://github.com/consideRatio)**
* [(#1312)](https://github.com/jupyterhub/zero-to-jupyterhub-k8s/issues/1312): _Gitlab Auth throw error_ by **[Jose3212](https://github.com/Jose3212)**


#### jupyterlab

##### [jupyterlab/jupyterlab](https://github.com/jupyterlab/jupyterlab)
* [(#6572)](https://github.com/jupyterlab/jupyterlab/issues/6572): _Ship vega4-extension by default_ by **[davidanthoff](https://github.com/davidanthoff)**
* [(#6560)](https://github.com/jupyterlab/jupyterlab/issues/6560): _Extension breaking after upgrading from 0.35.5 to 1.0.0a8 _ by **[tugceakin](https://github.com/tugceakin)**
* [(#6402)](https://github.com/jupyterlab/jupyterlab/issues/6402): _How to keep bundled extensions up-to-date_ by **[jakevdp](https://github.com/jakevdp)**
* [(#6380)](https://github.com/jupyterlab/jupyterlab/issues/6380): _How do you engineer a JupyterLab React Component?_ by **[bionicles](https://github.com/bionicles)**
* [(#6491)](https://github.com/jupyterlab/jupyterlab/issues/6491): _Support for yarn alias_ by **[gnestor](https://github.com/gnestor)**
* [(#6504)](https://github.com/jupyterlab/jupyterlab/issues/6504): _JupyterLab 1.0 Release Plan_ by **[jasongrout](https://github.com/jasongrout)**
* [(#6434)](https://github.com/jupyterlab/jupyterlab/issues/6434): _Disable (core) extensions via conf.d files_ by **[vidartf](https://github.com/vidartf)**
* [(#6545)](https://github.com/jupyterlab/jupyterlab/issues/6545): _Losing output after disconnecting from remote_ by **[cossio](https://github.com/cossio)**
* [(#6658)](https://github.com/jupyterlab/jupyterlab/issues/6658): _Downloading an svg file using the file browser fails_ by **[ellisonbg](https://github.com/ellisonbg)**
* [(#6537)](https://github.com/jupyterlab/jupyterlab/issues/6537): _Open directory from Path (manually cd)_ by **[tslaton](https://github.com/tslaton)**

##### [jupyterlab/jupyterlab-data-explorer](https://github.com/jupyterlab/jupyterlab-data-explorer)
* [(#27)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/27): _Filtering and Searching_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#26)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/26): _Add documentation_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#22)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/22): _Allow querying relative URL_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#21)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/21): _Named outputs from notebooks_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#20)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/20): _Vega viewer use data explorer to locate data_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#19)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/19): _Voyager Converter_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#18)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/18): _Provide data converters as mimerenders_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#17)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/17): _Improve expand/collapse UI_ by **[saulshanabrook](https://github.com/saulshanabrook)**
* [(#16)](https://github.com/jupyterlab/jupyterlab-data-explorer/issues/16): _View Notebook Ouputs in Data Explorer_ by **[saulshanabrook](https://github.com/saulshanabrook)**

##### [jupyterlab/jupyterlab-google-drive](https://github.com/jupyterlab/jupyterlab-google-drive)
* [(#157)](https://github.com/jupyterlab/jupyterlab-google-drive/issues/157): _Google Oauth Setup procedure does not work anymore (at least for Europe?)_ by **[jpdus](https://github.com/jpdus)**

##### [jupyterlab/jupyterlab-telemetry](https://github.com/jupyterlab/jupyterlab-telemetry)
* [(#6)](https://github.com/jupyterlab/jupyterlab-telemetry/issues/6): _Is this extension ready to be installed?_ by **[Econ808](https://github.com/Econ808)**
* [(#5)](https://github.com/jupyterlab/jupyterlab-telemetry/issues/5): _Move to jupyter/telemetry_ by **[Zsailer](https://github.com/Zsailer)**

